In [ ]:
import scipy.io as sio
import numpy as np
from matplotlib.colors import ListedColormap
from scipy.stats import pearsonr
import seaborn as sn
import mne
import matplotlib.pyplot as plt
%matplotlib qt
%gui qt

# File names for the baseline and signal files
str_FileName_R = 'Sujeto35_R_processed.fif'  # Name of the baseline file
str_FileName_MF = 'Sujeto35_MF_processed.fif'  # Name of the signal file
fileR = mne.read_epochs(str_FileName_R)  
fileMF = mne.read_epochs(str_FileName_MF)

# Get the data 
data_R = fileR.get_data()  # Shape: (n_epochs, n_channels, n_times)
data_MF = fileMF.get_data()
normalized_data = np.zeros_like(data_MF)

# Randomly select an epoch from the data (you can use np.random.choice to pick a random index)
random_epoch_index_R = np.random.randint(data_R.shape[0])  # Random index for fileR
random_epoch_index_MF = np.random.randint(data_MF.shape[0])  # Random index for fileMF

# Get the selected epoch
random_epoch_R = data_R[random_epoch_index_R]  # Shape: (n_channels, n_times)
random_epoch_MF = data_MF[random_epoch_index_MF]  # Shape: (n_channels, n_times)

# Get the number of time points for the random epoch
n_times_random_R = random_epoch_R.shape[1]  # Number of time points in the selected epoch
n_times_random_MF = random_epoch_MF.shape[1]  # Number of time points in the selected epoch

# Print the number of time points for each random epoch
print(f"Number of time points in the randomly selected epoch from fileR: {n_times_random_R}")
print(f"Number of time points in the randomly selected epoch from fileMF: {n_times_random_MF}")

## PSD in two different ways 

In [ ]:
###Plot signal and PSD

import numpy as np
import mne
import matplotlib.pyplot as plt

#fileR.plot(n_epochs=1, picks="eeg")
# fileMF.plot(n_epochs=1, picks="eeg", scalings="auto")

# Get the PSD data for both conditions
psd_R, freqs_R = fileR.compute_psd(method='welch', fmin=0.5, fmax=50 ).get_data(return_freqs=True)
psd_MF, freqs_MF = fileMF.compute_psd(method='welch', fmin=0.5, fmax=50).get_data(return_freqs=True)

# Number of channels
n_epochs_R, n_channels_R, _ = psd_R.shape
n_epochs_MF, n_channels_MF, _ = psd_MF.shape

# Plotting
fig, axs = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

channel_names = fileR.ch_names
# Plot averaged PSD for each channel in the Resting condition
for ch in range(n_channels_R):  # Only first 4 channels for better visualization
    avg_psd_R = np.mean(psd_R[:, ch, :], axis=0)/1e-12  # Average across epochs
    axs[0].plot(freqs_R, avg_psd_R, label=channel_names[ch])

# Plot averaged PSD for each channel in the MF condition
for ch in range(n_channels_MF):  # Only first 4 channels for better visualization
    avg_psd_MF = np.mean(psd_MF[:, ch, :], axis=0)/1e-12  # Average across epochs
    axs[1].plot(freqs_MF, avg_psd_MF, label=channel_names[ch])

axs[0].set_title("PSD - Resting Condition")
axs[1].set_title("PSD - MF Condition")
axs[1].set_xlabel("Frequency (Hz)")
axs[0].set_ylabel("Power Spectral Density (dB)")
axs[1].set_ylabel("Power Spectral Density (dB)")

# Show legend with channel names
axs[0].legend(loc="upper right", fontsize=8)
axs[1].legend(loc="upper right", fontsize=8)

plt.tight_layout()
plt.show()

In [ ]:
##Testing fft

import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft

# Assuming fileR.get_data() returns a 3D array (epochs, channels, time)
data = fileMF.get_data()  # 3D array of shape (epochs, channels, time)

# Extract the first channel across all epochs (shape: epochs, time)
data = data[:, 7, :]  

# Number of samples (time points) in each epoch
N = data.shape[1]  # Assuming the second dimension is the time (shape: epochs, time)
sampling_rate = 1024  # Sampling frequency in Hz

# Initialize a list to store the PSDs of each epoch
psds = []

# Iterate over each epoch
for epoch_data in data:
    # Perform the FFT for each epoch (only the first channel, which is already extracted)
    fft_result = fft(epoch_data)

    # Calculate the Power Spectral Density (PSD) as the squared magnitude of the FFT result
    psd = np.abs(fft_result) ** 2

    # Append the PSD to the list
    psds.append(psd)

# Convert the list of PSDs to a numpy array (shape: epochs, frequency bins)
psds = np.array(psds)

# Average the PSDs across all epochs
averaged_psd = np.mean(psds, axis=0)

# Calculate the corresponding frequency values
frequencies = np.fft.fftfreq(N, 1 / sampling_rate)

# Plot the averaged PSD (only positive frequencies)
plt.figure(figsize=(10, 6))
plt.plot(frequencies[:N // 2], averaged_psd[:N // 2])  # Plot the positive frequencies
plt.xlabel('Frequency (Hz)')
plt.ylabel('Average Power Spectral Density')
plt.title('Averaged PSD for First Channel Across Epochs')
plt.grid(True)
plt.xlim(0,50)
plt.show()

### Topomaps

In [ ]:
import numpy as np
import mne
import matplotlib.pyplot as plt

# Compute PSDs using Welch's method
psd_R, freqs_R = fileR.compute_psd(method='welch', fmin=0.5, fmax=30).get_data(return_freqs=True)
psd_MF, freqs_MF = fileMF.compute_psd(method='welch', fmin=0.5, fmax=30).get_data(return_freqs=True)

# Average PSD across all epochs
avg_psd_R = np.mean(psd_R, axis=0)  # Shape: (n_channels, n_freqs)
avg_psd_MF = np.mean(psd_MF, axis=0)  # Shape: (n_channels, n_freqs)

# Define frequency bands
frequency_bands = {
    'Delta': (0.5, 4),
    'Theta': (4, 8),
    'Alpha': (8, 12),
    'Beta': (12, 18),
    'Fast Beta': (18, 30),
}

# Plot topomaps for resting state
info = fileR.info  # Use the baseline's info for channel locations
n_bands = len(frequency_bands)
fig, axes = plt.subplots(1, n_bands, figsize=(15, 5))  # Create a row of subplots

for ax, (band, (fmin, fmax)) in zip(axes, frequency_bands.items()):
    # Find indices for the current band
    band_idx = np.logical_and(freqs_R >= fmin, freqs_R <= fmax)
    
    # Average the power across the band
    band_power_R = np.mean(avg_psd_R[:, band_idx], axis=1)  # Mean over the frequency axis
    
    # Plot the topomap
    im, _ = mne.viz.plot_topomap(
        band_power_R, 
        info, 
        axes=ax, 
        cmap='jet', 
        show=False  # Disable immediate showing to allow subplot arrangement
    )
    ax.set_title(f'{band} Band (Resting)')
    cbar = plt.colorbar(im, ax=ax, shrink=0.6, orientation='horizontal', pad=0.1)
    cbar.set_label('Power')
    
plt.suptitle("Resting State Topomap", fontsize=16)
plt.tight_layout()
plt.show()

# Plot topomaps for mindfulness state
fig, axes = plt.subplots(1, n_bands, figsize=(15, 5))  # Create a row of subplots

for ax, (band, (fmin, fmax)) in zip(axes, frequency_bands.items()):
    # Find indices for the current band
    band_idx = np.logical_and(freqs_MF >= fmin, freqs_MF <= fmax)
    
    # Average the power across the band
    band_power_MF = np.mean(avg_psd_MF[:, band_idx], axis=1)  # Mean over the frequency axis
    
    # Plot the topomap
    im, _ = mne.viz.plot_topomap(
        band_power_MF, 
        info, 
        axes=ax, 
        cmap='jet', 
        show=False  # Disable immediate showing to allow subplot arrangement
    )
    ax.set_title(f'{band} Band (Mindfulness)')
    cbar = plt.colorbar(im, ax=ax, shrink=0.6, orientation='horizontal', pad=0.1)
    cbar.set_label('Power')

plt.suptitle("Mindfulness State Topomap", fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
#Testing other function

import mne
import numpy as np
import matplotlib.pyplot as plt

# Define frequency range and FFT parameters
fmin, fmax = 0.5, 50  # Frequency range
n_fft = 256  # FFT length
tmax = 2.0  # Maximum time window for epochs
bands = [(1, 4, "Delta"), (4, 8, "Theta"), (8, 12, "Alpha"), (12, 18, "Beta slow"),(18, 30, "Beta fast"), (30, 50, "Gammma")]  # Frequency bands

# --- 1. PSD Plot for Raw Data ---
psd_R = fileR.compute_psd(method='welch',fmin=fmin, fmax=fmax)
psd_R.plot(dB=True, ci="std")

psd_MF = fileMF.compute_psd(method='welch',fmin=fmin, fmax=fmax)
psd_MF.plot(dB=True, ci="std")


# ####
# fig, axs = plt.subplots(3, 1)

# # Plot for psd_values_R in the first subplot (top left)
# psd_values_R, freqs = psd_R.get_data(return_freqs=True)
# for psd_value in psd_values_R:
#     axs[0].plot(freqs, 10 * np.log10(psd_value.mean(axis=0) / 1e-12))  # Plot the mean PSD across channels
# axs[0].set_xlabel("Frequency (Hz)")
# axs[0].set_ylabel("PSD (dB)")
# axs[0].set_title("PSD for psd_R")

# # Plot for psd_values_MF in the second subplot (top right)
# psd_values_MF, freqs = psd_MF.get_data(return_freqs=True)
# for psd_value in psd_values_MF:
#     axs[1].plot(freqs, 10 * np.log10(psd_value.mean(axis=0) / 1e-12))  # Plot the mean PSD across channels
# axs[1].set_xlabel("Frequency (Hz)")
# axs[1].set_ylabel("PSD (dB)")
# axs[1].set_title("PSD for psd_MF")

# # Calculate the difference (MF - R) and plot it in the third subplot (bottom left)
# for psd_value_R, psd_value_MF in zip(psd_values_R, psd_values_MF):
#     difference = 10 * np.log10(psd_value_MF.mean(axis=0) / 1e-12) - 10 * np.log10(psd_value_R.mean(axis=0) / 1e-12)
#     axs[2].plot(freqs, difference)  # Plot the difference in PSD (dB)
# axs[2].set_xlabel("Frequency (Hz)")
# axs[2].set_ylabel("Difference (dB)")
# axs[2].set_title("Difference (MF - R)")

# plt.tight_layout()
# plt.show()
# ###


# --- 2. Topomap of PSD for Epochs ---
# print(epochs_psd_R.get_data())
psd_R.plot_topomap(bands=bands, normalize=True, cmap='jet')
psd_MF.plot_topomap(bands=bands, normalize=True, cmap='jet')


# Get PSD data
psd_values_R, freqs = psd_R.get_data(return_freqs=True)
psd_values_MF, _ = psd_MF.get_data(return_freqs=True)

# Compute mean PSD across trials
mean_psd_R = np.mean(psd_values_R, axis=0)  # Shape: (n_channels, n_freqs)
mean_psd_MF = np.mean(psd_values_MF, axis=0)

# Compute the difference in (MF - R)
psd_diff = (mean_psd_MF / 1e-12) 

# Define frequency bands
bands = [(1, 4, "Delta"), (4, 8, "Theta"), (8, 12, "Alpha"),
         (12, 18, "Beta slow"), (18, 30, "Beta fast"), (30, 50, "Gamma")]

# Create topoplots for each band
fig, axes = plt.subplots(1, 6)  # Adjust grid for 6 plots


for ax, (fmin, fmax, name) in zip(axes.flatten(), bands):
    freq_idx = np.where((freqs >= fmin) & (freqs <= fmax))[0]
    psd_diff_band = psd_diff[:, freq_idx].mean(axis=1)  # Average over selected frequencies
    
    # Plot topomap
    im, _ = mne.viz.plot_topomap(psd_diff_band, psd_R.info, axes=ax, show=False, cmap="jet")
    ax.set_title(f"{name} [{fmin}-{fmax} Hz]")

# Add colorbar
fig.colorbar(im, ax=axes.ravel().tolist(), shrink=0.5)
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import mne
import matplotlib.pyplot as plt

# Compute PSDs using Welch's method
psd_R, freqs_R = fileR.compute_psd(method='multitaper', fmin=0.5, fmax=50).get_data(return_freqs=True)
psd_MF, freqs_MF = fileMF.compute_psd(method='multitaper', fmin=0.5, fmax=50).get_data(return_freqs=True)

# Average PSD across all windows (mean over epochs axis, axis=0)
avg_psd_R = np.mean(psd_R, axis=0)  # Shape: (n_channels, n_freqs)
avg_psd_MF = np.mean(psd_MF, axis=0)  # Shape: (n_channels, n_freqs)

# Calculate the difference
difference_psd = (avg_psd_MF - avg_psd_R)/1e-12  # Shape: (n_channels, n_freqs)

# Define frequency bands
frequency_bands = {
    'Delta': (0.5, 4),
    'Theta': (4, 8),
    'Alpha': (8, 12),
    'Beta': (12, 18),
    'Fast Beta': (18, 30),
    'Gamma': (30, 50)
}

# Calculate global min and max across all bands
global_min = np.inf
global_max = -np.inf

for fmin, fmax in frequency_bands.values():
    band_idx = np.logical_and(freqs_R >= fmin, freqs_R <= fmax)
    band_power_difference = np.mean(difference_psd[:, band_idx], axis=1)  # Mean over frequencies
    global_min = min(global_min, band_power_difference.min())
    global_max = max(global_max, band_power_difference.max())

# Extract band-specific differences and prepare the figure
info = fileR.info  # Use the baseline's info for channel locations
n_bands = len(frequency_bands)
fig, axes = plt.subplots(1, n_bands, figsize=(15, 5))  # Create a row of subplots

for ax, (band, (fmin, fmax)) in zip(axes, frequency_bands.items()):
    # Find indices for the current band
    band_idx = np.logical_and(freqs_R >= fmin, freqs_R <= fmax)
    
    # Average the difference across the band
    band_power_difference = np.mean(difference_psd[:, band_idx], axis=1)  # Mean over the frequency axis
    
    # Plot the topomap with consistent vlim
    im, _ = mne.viz.plot_topomap(
        band_power_difference, 
        info, 
        axes=ax, 
        cmap='jet', 
        show=False,  # Disable immediate showing to allow subplot arrangement
        names=fileR.ch_names,
    #   vlim=(global_min, global_max)  # Use global min and max
    )
    ax.set_title(f"{band} Band")
    # Add a colorbar for the subplot
    cbar = plt.colorbar(im, ax=ax, shrink=0.6, orientation='horizontal', pad=0.1)
    cbar.set_label('Power Difference')

# Finalize and display the figure
plt.suptitle("Mindfulness Difference", fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# import numpy as np
# import mne
# import matplotlib.pyplot as plt

# # Compute PSDs using Welch's method
# psd_R, freqs_R = fileR.compute_psd(method='welch', fmin=0.5, fmax=30).get_data(return_freqs=True)
# psd_MF, freqs_MF = fileMF.compute_psd(method='welch', fmin=0.5, fmax=30).get_data(return_freqs=True)

# # Normalize PSDs (relative power for each channel)
# total_power_R = np.sum(psd_R, axis=2, keepdims=True)  # Total power across frequencies per channel for R
# total_power_MF = np.sum(psd_MF, axis=2, keepdims=True)  # Total power across frequencies per channel for MF
# relative_psd_R = psd_R / total_power_R  # Shape: (n_epochs, n_channels, n_freqs)
# relative_psd_MF = psd_MF / total_power_MF

# # Average relative PSDs across epochs
# avg_relative_psd_R = np.mean(relative_psd_R, axis=0)  # Shape: (n_channels, n_freqs)
# avg_relative_psd_MF = np.mean(relative_psd_MF, axis=0)  # Shape: (n_channels, n_freqs)

# # Compute the relative power difference
# relative_power_difference = avg_relative_psd_MF - avg_relative_psd_R  # Shape: (n_channels, n_freqs)

# # Define frequency bands
# frequency_bands = {
#     'Delta': (0.5, 4),
#     'Theta': (4, 8),
#     'Alpha': (8, 12),
#     'Beta': (12, 18),
#     'FastBeta': (18, 30),
# }

# # Prepare the figure for topomaps
# info = fileR.info  # Use the baseline's info for channel locations
# n_bands = len(frequency_bands)
# fig, axes = plt.subplots(1, n_bands, figsize=(15, 5))  # Create a row of subplots

# for ax, (band, (fmin, fmax)) in zip(axes, frequency_bands.items()):
#     # Find indices for the current band
#     band_idx = np.logical_and(freqs_R >= fmin, freqs_R <= fmax)
    
#     # Average the relative power difference across the band
#     band_relative_power_diff = np.mean(relative_power_difference[:, band_idx], axis=1)  # Mean over frequencies
   
#     # Plot the topomap with a colorbar
#     im, _ = mne.viz.plot_topomap(
#         band_relative_power_diff, 
#         info, 
#         axes=ax, 
#         cmap='jet', 
#         show=False  # Disable immediate showing to allow subplot arrangement
#     )
#     ax.set_title(f'{band} Band')
    
#     # Add a colorbar for the subplot
#     cbar = plt.colorbar(im, ax=ax, shrink=0.6, orientation='horizontal', pad=0.1)
#     cbar.set_label('Relative Power Difference')

# # Finalize and display the figure
# plt.tight_layout()
# plt.show()

## Resting, Mind, DiffNormalized

In [ ]:
import numpy as np
import mne
import matplotlib.pyplot as plt

# Compute PSDs using Welch's method
psd_R, freqs_R = fileR.compute_psd(method='welch', fmin=0.5, fmax=50).get_data(return_freqs=True)
psd_MF, freqs_MF = fileMF.compute_psd(method='welch', fmin=0.5, fmax=50).get_data(return_freqs=True)

# Normalize PSDs (relative power for each channel)
total_power_R = np.sum(psd_R, axis=2, keepdims=True)  # Total power across frequencies per channel for R
total_power_MF = np.sum(psd_MF, axis=2, keepdims=True)  # Total power across frequencies per channel for MF
relative_psd_R = psd_R / total_power_R  # Shape: (n_epochs, n_channels, n_freqs)
relative_psd_MF = psd_MF / total_power_MF

# Average relative PSDs across epochs
avg_relative_psd_R = np.mean(relative_psd_R, axis=0)   # Shape: (n_channels, n_freqs)
avg_relative_psd_MF = np.mean(relative_psd_MF, axis=0)   # Shape: (n_channels, n_freqs)

# Compute the relative power difference
relative_power_difference = avg_relative_psd_MF - avg_relative_psd_R  # Shape: (n_channels, n_freqs)

# Define frequency bands
frequency_bands = {
    'Delta': (0.5, 4),
    'Theta': (4, 8),
    'Alpha': (8, 12),
    'Beta': (12, 18),
    'FastBeta': (18, 30),
    'Gamma': (30, 50)
}

# Prepare the figure for topomaps
info = fileR.info  # Use the baseline's info for channel locations
n_bands = len(frequency_bands)
fig, axes = plt.subplots(3, n_bands, figsize=(11, 12),gridspec_kw={'wspace': 0.0001, 'hspace': 0.0001})  # Adjust spacing  # Create a grid of subplots

for row, data, title in zip(
    range(3),
    [avg_relative_psd_R, avg_relative_psd_MF, relative_power_difference],
    ['Resting', 'Mindfulness', 'Normalized Difference']
):
    for ax, (band, (fmin, fmax)) in zip(axes[row], frequency_bands.items()):
        # Find indices for the current band
        band_idx = np.logical_and(freqs_R >= fmin, freqs_R <= fmax)
        
        # Average the data across the band
        band_data = np.mean(data[:, band_idx], axis=1)  # Mean over frequencies
        
        # Plot the topomap with a colorbar
        im, _ = mne.viz.plot_topomap(
            band_data, 
            info, 
            axes=ax, 
            cmap='jet', 
            show=False,  # Disable immediate showing to allow subplot arrangement
        )
        ax.set_title(f'{band} Band' if row == 0 else '')
        
        # Add a colorbar for the subplot
        cbar = plt.colorbar(im, ax=ax, shrink=0.6, orientation='horizontal', pad=0.1)
        if row == 2:  # Label the colorbar only for the last row
            cbar.set_label(title)

# Add row titles
for ax, title in zip(axes[:, 0], ['Resting', 'Mindfulness', 'Normalized Difference']):
    ax.annotate(
        title, 
        xy=(0, 0.5), 
        xytext=(-ax.yaxis.labelpad - 5, 0),
        xycoords=ax.yaxis.label, 
        textcoords='offset points',
        size='large', ha='right', va='center', rotation=90
    )

# Finalize and display the figure
plt.tight_layout()
plt.show()

## For Several files (relative)

In [ ]:
def topoplotsAverage(file_pairs, fmin=0.5, fmax=50):
    """
    Compute and plot the average PSDs and their differences for multiple pairs of EEG files,
    ensuring each row has the same color scale while different rows have different scales.

    Parameters:
    - file_pairs: List of tuples [(fileR1, fileMF1), (fileR2, fileMF2), ...]
                  Each tuple contains a resting state file and a mindfulness state file.
    - fmin: Minimum frequency for PSD calculation (default: 0.5 Hz).
    - fmax: Maximum frequency for PSD calculation (default: 30 Hz).

    Returns:
    - None (plots the results).
    """
    avg_psd_R_all = []
    avg_psd_MF_all = []

    for str_FileName_R, str_FileName_MF in file_pairs:
        fileR = mne.read_epochs(str_FileName_R)  
        fileMF = mne.read_epochs(str_FileName_MF)

        print("######################################Shape of file R:", np.shape(fileR))
        print("######################################Shape of file MF:", np.shape(fileMF))
        # fileR.compute_psd(fmax=50, picks=['eeg']).plot()
        # fileMF.compute_psd(fmax=50, picks=['eeg']).plot()
        
        # Compute PSDs using Welch's method
        psd_R, freqs_R = fileR.compute_psd(method='welch', fmin=fmin, fmax=fmax).get_data(return_freqs=True)
        psd_MF, freqs_MF = fileMF.compute_psd(method='welch', fmin=fmin, fmax=fmax).get_data(return_freqs=True)

        # Normalize PSDs (relative power for each channel)
        total_power_R = np.sum(psd_R, axis=2, keepdims=True)
        total_power_MF = np.sum(psd_MF, axis=2, keepdims=True)
        # relative_psd_R = psd_R / total_power_R #add for for relative power
        # relative_psd_MF = psd_MF / total_power_MF #

        relative_psd_R = psd_R / 1e-12 ## For total power
        relative_psd_MF = psd_MF / 1e-12 # For total power

        # Average relative PSDs across epochs
        avg_relative_psd_R = np.mean(relative_psd_R, axis=0)
        avg_relative_psd_MF = np.mean(relative_psd_MF, axis=0)

        avg_psd_R_all.append(avg_relative_psd_R)
        avg_psd_MF_all.append(avg_relative_psd_MF)

        print("######################################Shape of file MF:", np.shape(avg_relative_psd_R))
        print("######################################Shape of file MF:", np.shape(avg_relative_psd_MF))

    # Compute grand averages across all pairs
    avg_psd_R_all = np.mean(avg_psd_R_all, axis=0)
    avg_psd_MF_all = np.mean(avg_psd_MF_all, axis=0)

    # Compute the relative power difference
    relative_power_difference = avg_psd_MF_all - avg_psd_R_all

    # Define frequency bands
    frequency_bands = {
        'Delta': (0.5, 4),
        'Theta': (4, 8),
        'Alpha': (8, 12),
        'Beta': (12, 18),
        'FastBeta': (18, 30),
        'Gamma': (30, 50)
    }

    # Prepare the figure for topomaps
    info = mne.read_epochs(file_pairs[0][0]).info  # Use the info from the first resting state file
    n_bands = len(frequency_bands)
    fig, axes = plt.subplots(3, n_bands, figsize=(11, 12), gridspec_kw={'wspace': 0.001, 'hspace': 0.001})

    for row, data, title in zip(
        range(3),
        [avg_psd_R_all, avg_psd_MF_all, relative_power_difference],
        ['Resting', 'Mindfulness', 'Normalized Difference']
    ):
        # Initialize lists to store the min and max values across the row
        row_min = []
        row_max = []
        
        for ax, (band, (fmin, fmax)) in zip(axes[row], frequency_bands.items()):
            # Find indices for the current band
            band_idx = np.logical_and(freqs_R >= fmin, freqs_R <= fmax)

            # Average the data across the band
            band_data = np.mean(data[:, band_idx], axis=1)

            # Find the min and max values for the current band
            band_min = band_data.min()
            band_max = band_data.max()

            # Store the min and max for the row
            row_min.append(band_min)
            row_max.append(band_max)

            # Plot the topomap without setting vlim yet
            im, _ = mne.viz.plot_topomap(
                band_data,
                info,
                axes=ax,
                cmap='jet',
                show=False,
            )
            ax.set_title(f'{band} Band' if row == 0 else '')

            # Add a colorbar for the subplot
            cbar = plt.colorbar(im, ax=ax, shrink=0.6, orientation='horizontal', pad=0.1)
            if row == 2:
                cbar.set_label(title)

        # After collecting all min and max values for the row, set the vlim for each subplot
        row_vmin = min(row_min)
        row_vmax = max(row_max)

        # Apply the same vlim for all subplots in the current row
        # for ax in axes[row]:
        #     im = ax.images[0]
        #     im.set_clim(vmin=row_vmin, vmax=row_vmax) #set the colorbar

    # Add row titles
    for ax, title in zip(axes[:, 0], ['Resting', 'Mindfulness', 'Normalized Difference']):
        ax.annotate(
            title,
            xy=(0, 0.5),
            xytext=(-ax.yaxis.labelpad - 5, 0),
            xycoords=ax.yaxis.label,
            textcoords='offset points',
            size='large', ha='right', va='center', rotation=90
        )

    # Finalize and display the figure
    plt.tight_layout()
    plt.show()

file_pairs = [
    ('Sujeto1_R_processed.fif', 'Sujeto1_MF_processed.fif'),
    ('Sujeto2_R_processed.fif', 'Sujeto2_MF_processed.fif'),
    ('Sujeto3_R_processed.fif', 'Sujeto3_MF_processed.fif'),
    ('Sujeto4_R_processed.fif', 'Sujeto4_MF_processed.fif'),
    ('Sujeto5_R_processed.fif', 'Sujeto5_MF_processed.fif'),
    ('Sujeto6_R_processed.fif', 'Sujeto6_MF_processed.fif'),
    ('Sujeto7_R_processed.fif', 'Sujeto7_MF_processed.fif'),
    ('Sujeto8_R_processed.fif', 'Sujeto8_MF_processed.fif'),
    ('Sujeto9_R_processed.fif', 'Sujeto9_MF_processed.fif'),
    ('Sujeto10_R_processed.fif', 'Sujeto10_MF_processed.fif'),
    ('Sujeto11_R_processed.fif', 'Sujeto11_MF_processed.fif'),
    # ('Sujeto12_R_processed.fif', 'Sujeto12_MF_processed.fif'),
    ('Sujeto13_R_processed.fif', 'Sujeto13_MF_processed.fif'),
    ('Sujeto14_R_processed.fif', 'Sujeto14_MF_processed.fif'),
    ('Sujeto15_R_processed.fif', 'Sujeto15_MF_processed.fif'),
    ('Sujeto16_R_processed.fif', 'Sujeto16_MF_processed.fif'),
    ('Sujeto17_R_processed.fif', 'Sujeto17_MF_processed.fif'),    
    ('Sujeto18_R_processed.fif', 'Sujeto18_MF_processed.fif'),
    ('Sujeto19_R_processed.fif', 'Sujeto19_MF_processed.fif'),
    ('Sujeto20_R_processed.fif', 'Sujeto20_MF_processed.fif'),
    ('Sujeto21_R_processed.fif', 'Sujeto21_MF_processed.fif'),
    ('Sujeto22_R_processed.fif', 'Sujeto22_MF_processed.fif'),
    ('Sujeto23_R_processed.fif', 'Sujeto23_MF_processed.fif'),
    ('Sujeto24_R_processed.fif', 'Sujeto24_MF_processed.fif'),
    ('Sujeto25_R_processed.fif', 'Sujeto25_MF_processed.fif'),
    ('Sujeto26_R_processed.fif', 'Sujeto26_MF_processed.fif'),
    ('Sujeto27_R_processed.fif', 'Sujeto27_MF_processed.fif'),
    ('Sujeto28_R_processed.fif', 'Sujeto28_MF_processed.fif'),
    ('Sujeto29_R_processed.fif', 'Sujeto29_MF_processed.fif'),
    ('Sujeto30_R_processed.fif', 'Sujeto30_MF_processed.fif'),
    ('Sujeto31_R_processed.fif', 'Sujeto31_MF_processed.fif'),
    ('Sujeto32_R_processed.fif', 'Sujeto32_MF_processed.fif'),
    ('Sujeto33_R_processed.fif', 'Sujeto33_MF_processed.fif'),
    ('Sujeto34_R_processed.fif', 'Sujeto34_MF_processed.fif'),
    #('Sujeto35_R_processed.fif', 'Sujeto35_MF_processed.fif'),
    ('Sujeto36_R_processed.fif', 'Sujeto36_MF_processed.fif'),
    ('Sujeto37_R_processed.fif', 'Sujeto37_MF_processed.fif')
]

topoplotsAverage(file_pairs)

## Topoplots with significance

In [18]:
import numpy as np
import matplotlib.pyplot as plt
import mne
from scipy.stats import permutation_test

def topoplots_with_significance(file_pairs, fmin=0.5, fmax=50, n_permutations=1000):
    """
    Compute and plot average PSD topomaps and significance maps using permutation tests.

    Parameters:
    - file_pairs: List of (resting_file, mindfulness_file) .fif epochs.
    - fmin, fmax: Frequency range for Welch PSD.
    - n_permutations: Number of permutations for significance test.

    Returns:
    - None (plots topomaps and significance results).
    """
    all_diff = []  # To store per-subject differences for all bands

    # Define frequency bands
    frequency_bands = {
        'Delta': (0.5, 4),
        'Theta': (4, 8),
        'Alpha': (8, 12),
        'Beta': (12, 18),
        'FastBeta': (18, 30),
        'Gamma': (30, 50)
    }

    band_names = list(frequency_bands.keys())
    n_bands = len(frequency_bands)

    # Use the info of the first file for plotting
    info = mne.read_epochs(file_pairs[0][0]).info

    # Initialize structure for all subjects and all bands
    n_channels = len(info['ch_names'])
    band_differences = {band: [] for band in band_names}

    for file_R, file_MF in file_pairs:
        epochs_R = mne.read_epochs(file_R)
        epochs_MF = mne.read_epochs(file_MF)

        psd_R, freqs = epochs_R.compute_psd(method='welch', fmin=fmin, fmax=fmax).get_data(return_freqs=True)
        psd_MF, _ = epochs_MF.compute_psd(method='welch', fmin=fmin, fmax=fmax).get_data(return_freqs=True)

        rel_R = psd_R / 1e-12
        rel_MF = psd_MF / 1e-12

        avg_rel_R = np.mean(rel_R, axis=0)
        avg_rel_MF = np.mean(rel_MF, axis=0)

        for band in band_names:
            band_min, band_max = frequency_bands[band]
            idx = np.logical_and(freqs >= band_min, freqs <= band_max)

            mean_band_R = np.mean(avg_rel_R[:, idx], axis=1)  # (n_channels,)
            mean_band_MF = np.mean(avg_rel_MF[:, idx], axis=1)

            diff = mean_band_MF - mean_band_R
            band_differences[band].append(diff)  # Append to band-specific list

    # Prepare plots
    fig, axes = plt.subplots(nrows=3, ncols=n_bands, figsize=(3 * n_bands, 9))

    for i, band in enumerate(band_names):
        data = np.array(band_differences[band])  # shape (n_subjects, n_channels)
        mean_diff = data.mean(axis=0)

        # Permutation test per channel
        p_vals = np.array([
            permutation_test((data[:, ch], np.zeros(data.shape[0])), 
                             statistic=lambda x, y: np.mean(x), 
                             n_resamples=n_permutations,
                             alternative='two-sided').pvalue
            for ch in range(n_channels)
        ])

        # Significance mask
        sig_mask = p_vals < 0.05
        sig_data = np.where(sig_mask, mean_diff, np.nan)

        # Plot 1: Difference
        im1, _ = mne.viz.plot_topomap(mean_diff, info, axes=axes[0, i], show=False, cmap='jet')
        axes[0, i].set_title(f"{band} diff")
        plt.colorbar(im1, ax=axes[0, i], orientation='horizontal', shrink=0.6)

        # Plot 2: p-value map
        im2, _ = mne.viz.plot_topomap(p_vals, info, axes=axes[1, i], show=False, cmap='magma_r')
        axes[1, i].set_title(f"{band} p-values")
        plt.colorbar(im2, ax=axes[1, i], orientation='horizontal', shrink=0.6)

        # Plot 3: Significant only
        im3, _ = mne.viz.plot_topomap(p_vals, info, axes=axes[2, i], show=False, cmap='seismic', vlim=(0,0.1))
        axes[2, i].set_title(f"{band} p<0.05")
        plt.colorbar(im3, ax=axes[2, i], orientation='horizontal', shrink=0.6)

    # Row labels
    for ax, label in zip(axes[:, 0], ['Mean Diff', 'P-value', 'Significant']):
        ax.annotate(label, xy=(-0.15, 0.5), xycoords='axes fraction', va='center', ha='right', rotation=90, fontsize=12)

    plt.tight_layout()
    plt.show()

file_pairs = [
    ('Sujeto1_R_processed.fif', 'Sujeto1_MF_processed.fif'),
    ('Sujeto2_R_processed.fif', 'Sujeto2_MF_processed.fif'),
    ('Sujeto3_R_processed.fif', 'Sujeto3_MF_processed.fif'),
    ('Sujeto4_R_processed.fif', 'Sujeto4_MF_processed.fif'),
    # ('Sujeto5_R_processed.fif', 'Sujeto5_MF_processed.fif'),
    # ('Sujeto6_R_processed.fif', 'Sujeto6_MF_processed.fif'),
    # ('Sujeto7_R_processed.fif', 'Sujeto7_MF_processed.fif'),
    # ('Sujeto8_R_processed.fif', 'Sujeto8_MF_processed.fif'),
    # ('Sujeto9_R_processed.fif', 'Sujeto9_MF_processed.fif'),
    # ('Sujeto10_R_processed.fif', 'Sujeto10_MF_processed.fif'),
    # ('Sujeto11_R_processed.fif', 'Sujeto11_MF_processed.fif'),
    # #('Sujeto12_R_processed.fif', 'Sujeto12_MF_processed.fif'),
    # ('Sujeto13_R_processed.fif', 'Sujeto13_MF_processed.fif'),
    # ('Sujeto14_R_processed.fif', 'Sujeto14_MF_processed.fif'),
    # ('Sujeto15_R_processed.fif', 'Sujeto15_MF_processed.fif'),
    # ('Sujeto16_R_processed.fif', 'Sujeto16_MF_processed.fif'),
    # ('Sujeto17_R_processed.fif', 'Sujeto17_MF_processed.fif'),    
    # ('Sujeto18_R_processed.fif', 'Sujeto18_MF_processed.fif'),
    # ('Sujeto19_R_processed.fif', 'Sujeto19_MF_processed.fif'),
    # ('Sujeto20_R_processed.fif', 'Sujeto20_MF_processed.fif'),
    # ('Sujeto21_R_processed.fif', 'Sujeto21_MF_processed.fif'),
    # ('Sujeto22_R_processed.fif', 'Sujeto22_MF_processed.fif'),
    # ('Sujeto23_R_processed.fif', 'Sujeto23_MF_processed.fif'),
    # ('Sujeto24_R_processed.fif', 'Sujeto24_MF_processed.fif'),
    # ('Sujeto25_R_processed.fif', 'Sujeto25_MF_processed.fif'),
    # ('Sujeto26_R_processed.fif', 'Sujeto26_MF_processed.fif'),
    # ('Sujeto27_R_processed.fif', 'Sujeto27_MF_processed.fif'),
    # ('Sujeto28_R_processed.fif', 'Sujeto28_MF_processed.fif'),
    # ('Sujeto29_R_processed.fif', 'Sujeto29_MF_processed.fif'),
    # ('Sujeto30_R_processed.fif', 'Sujeto30_MF_processed.fif'),
    # ('Sujeto31_R_processed.fif', 'Sujeto31_MF_processed.fif'),
    # ('Sujeto32_R_processed.fif', 'Sujeto32_MF_processed.fif'),
    # ('Sujeto33_R_processed.fif', 'Sujeto33_MF_processed.fif'),
    # ('Sujeto34_R_processed.fif', 'Sujeto34_MF_processed.fif'),
    # #('Sujeto35_R_processed.fif', 'Sujeto35_MF_processed.fif'),
    # ('Sujeto36_R_processed.fif', 'Sujeto36_MF_processed.fif'),
    # ('Sujeto37_R_processed.fif', 'Sujeto37_MF_processed.fif')
]

topoplots_with_significance(file_pairs)

Reading c:\Users\afcad\Downloads\Sujeto1_R_processed.fif ...


    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available


C:\Users\afcad\AppData\Local\Temp\ipykernel_1044\2668377417.py:34: RuntimeWarning: This filename (Sujeto1_R_processed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  info = mne.read_epochs(file_pairs[0][0]).info


Not setting metadata
68 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\afcad\Downloads\Sujeto1_R_processed.fif ...
    Found the data of interest:


C:\Users\afcad\AppData\Local\Temp\ipykernel_1044\2668377417.py:41: RuntimeWarning: This filename (Sujeto1_R_processed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs_R = mne.read_epochs(file_R)


...

Not setting metadata
131 matching events found
No baseline correction applied
0 projection items activated
Effective window size : 0.250 (s)
Effective window size : 0.250 (s)
Reading c:\Users\afcad\Downloads\Sujeto4_R_processed.fif ...
    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available


C:\Users\afcad\AppData\Local\Temp\ipykernel_1044\2668377417.py:41: RuntimeWarning: This filename (Sujeto4_R_processed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs_R = mne.read_epochs(file_R)


Not setting metadata
60 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\afcad\Downloads\Sujeto4_MF_processed.fif ...
    Found the data of interest:


C:\Users\afcad\AppData\Local\Temp\ipykernel_1044\2668377417.py:42: RuntimeWarning: This filename (Sujeto4_MF_processed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs_MF = mne.read_epochs(file_MF)


        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available
Not setting metadata
143 matching events found
No baseline correction applied
0 projection items activated
Effective window size : 0.250 (s)
Effective window size : 0.250 (s)


In [ ]:
import random

# Generate 10 random combinations of 5 pairs
random_combinations = [random.sample(file_pairs, 20) for _ in range(20)]

# Print the combinations
for i, combination in enumerate(random_combinations, 1):
    print(f"Combination {i}: {combination}")
    topoplots_with_significance(combination)

## Z-score

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# import mne
# from scipy.stats import permutation_test, zscore

# def topoplots_with_zscore_and_significance(file_pairs, fmin=0.5, fmax=50, n_permutations=1000):
#     """
#     Compute and plot z-score topomaps with significant electrodes marked (p < 0.05)
#     """
#     # Define frequency bands
#     frequency_bands = {
#         'Delta': (0.5, 4),
#         'Theta': (4, 8),
#         'Alpha': (8, 12),
#         'Beta': (12, 18),
#         'FastBeta': (18, 30),
#         'Gamma': (30, 50)
#     }

#     band_names = list(frequency_bands.keys())
#     n_bands = len(frequency_bands)

#     # Load info from first file
#     info = mne.read_epochs(file_pairs[0][0]).info
#     n_channels = len(info['ch_names'])

#     # Initialize data storage
#     band_data = {band: {'rest': [], 'mind': []} for band in band_names}

#     # Process all files
#     for file_R, file_MF in file_pairs:
#         epochs_R = mne.read_epochs(file_R)
#         epochs_MF = mne.read_epochs(file_MF)

#         # Compute PSD
#         psd_R, freqs = epochs_R.compute_psd(method='welch', fmin=fmin, fmax=fmax).get_data(return_freqs=True)
#         psd_MF, _ = epochs_MF.compute_psd(method='welch', fmin=fmin, fmax=fmax).get_data(return_freqs=True)

#         # Convert to relative power
#         rel_R = psd_R / 1e-12
#         rel_MF = psd_MF / 1e-12

#         # Average across epochs
#         avg_rel_R = np.mean(rel_R, axis=0)
#         avg_rel_MF = np.mean(rel_MF, axis=0)

#         # Process each band
#         for band in band_names:
#             band_min, band_max = frequency_bands[band]
#             idx = np.logical_and(freqs >= band_min, freqs <= band_max)

#             mean_band_R = np.mean(avg_rel_R[:, idx], axis=1)
#             mean_band_MF = np.mean(avg_rel_MF[:, idx], axis=1)

#             band_data[band]['rest'].append(mean_band_R)
#             band_data[band]['mind'].append(mean_band_MF)

#     # Create figure
#     fig, axes = plt.subplots(2, n_bands, figsize=(3*n_bands, 6))
#     plt.suptitle("Z-scores with significant electrodes (p < 0.05) marked", y=1.02)

#     # Plot each band
#     for i, band in enumerate(band_names):
#         rest_data = np.array(band_data[band]['rest'])
#         mind_data = np.array(band_data[band]['mind'])

#         # Calculate z-scores
#         rest_z = zscore(rest_data, axis=0)
#         mind_z = (mind_data - np.mean(rest_data, axis=0)) / np.std(rest_data, axis=0, ddof=1)

#         # Average across subjects
#         mean_rest_z = np.mean(rest_z, axis=0)
#         mean_mind_z = np.mean(mind_z, axis=0)

#         # Permutation test
#         p_vals = np.array([
#             permutation_test(
#                 (rest_data[:, ch], mind_data[:, ch]),
#                 statistic=lambda x, y: np.mean(y) - np.mean(x),
#                 n_resamples=n_permutations,
#                 alternative='two-sided'
#             ).pvalue
#             for ch in range(n_channels)
#         ])

#         # Get significant channels
#         sig_mask = p_vals < 0.05
#         sig_ch_names = [info['ch_names'][i] for i in np.where(sig_mask)[0]]

#         # Plot resting state
#         im = mne.viz.plot_topomap(
#             mean_rest_z, info, axes=axes[0,i], show=False, 
#             cmap='RdBu_r'
#         )
#         axes[0,i].set_title(f"{band}\nResting Z")
#         plt.colorbar(im[0], ax=axes[0,i], orientation='horizontal', shrink=0.6)

#         # Plot mindfulness state
#         im = mne.viz.plot_topomap(
#             mean_mind_z, info, axes=axes[1,i], show=False, 
#             cmap='RdBu_r'
#         )
#         axes[1,i].set_title(f"{band}\nMindfulness Z")
#         plt.colorbar(im[0], ax=axes[1,i], orientation='horizontal', shrink=0.6)

#         # Mark significant electrodes on both plots
#         if len(sig_ch_names) > 0:
#             # Get positions of significant channels
#             pos = mne.viz.topomap._prepare_topomap_plot(info, 'eeg')[0]
#             sig_idx = [i for i, ch in enumerate(info['ch_names']) if ch in sig_ch_names]
#             x, y = pos[sig_idx, 0], pos[sig_idx, 1]

#             # Add markers
#             for ax in [axes[0,i], axes[1,i]]:
#                 ax.scatter(x, y, s=100, facecolors='none', edgecolors='k', linewidths=1.5)
#                 ax.scatter(x, y, s=30, c='k', alpha=0.7)

#     # Add row labels
#     for ax, label in zip(axes[:,0], ['Resting Z', 'Mindfulness Z']):
#         ax.annotate(
#             label, xy=(-0.2, 0.5), xycoords='axes fraction',
#             va='center', ha='right', rotation=90, fontsize=12
#         )

#     plt.tight_layout()
#     plt.show()

# # Example usage
# file_pairs = [
#     ('Sujeto1_R_processed.fif', 'Sujeto1_MF_processed.fif'),
#     ('Sujeto2_R_processed.fif', 'Sujeto2_MF_processed.fif'),
#     ('Sujeto3_R_processed.fif', 'Sujeto3_MF_processed.fif'),
#     ('Sujeto4_R_processed.fif', 'Sujeto4_MF_processed.fif'),
#     ('Sujeto5_R_processed.fif', 'Sujeto5_MF_processed.fif'),
#     ('Sujeto6_R_processed.fif', 'Sujeto6_MF_processed.fif'),
#     ('Sujeto7_R_processed.fif', 'Sujeto7_MF_processed.fif'),
#     ('Sujeto8_R_processed.fif', 'Sujeto8_MF_processed.fif'),
#     ('Sujeto9_R_processed.fif', 'Sujeto9_MF_processed.fif'),
#     ('Sujeto10_R_processed.fif', 'Sujeto10_MF_processed.fif'),
#     ('Sujeto11_R_processed.fif', 'Sujeto11_MF_processed.fif'),
#     #('Sujeto12_R_processed.fif', 'Sujeto12_MF_processed.fif'),
#     ('Sujeto13_R_processed.fif', 'Sujeto13_MF_processed.fif'),
#     ('Sujeto14_R_processed.fif', 'Sujeto14_MF_processed.fif'),
#     ('Sujeto15_R_processed.fif', 'Sujeto15_MF_processed.fif'),
#     ('Sujeto16_R_processed.fif', 'Sujeto16_MF_processed.fif'),
#     ('Sujeto17_R_processed.fif', 'Sujeto17_MF_processed.fif'),    
#     ('Sujeto18_R_processed.fif', 'Sujeto18_MF_processed.fif'),
#     ('Sujeto19_R_processed.fif', 'Sujeto19_MF_processed.fif'),
#     ('Sujeto20_R_processed.fif', 'Sujeto20_MF_processed.fif'),
#     ('Sujeto21_R_processed.fif', 'Sujeto21_MF_processed.fif'),
#     ('Sujeto22_R_processed.fif', 'Sujeto22_MF_processed.fif'),
#     ('Sujeto23_R_processed.fif', 'Sujeto23_MF_processed.fif'),
#     ('Sujeto24_R_processed.fif', 'Sujeto24_MF_processed.fif'),
#     ('Sujeto25_R_processed.fif', 'Sujeto25_MF_processed.fif'),
#     ('Sujeto26_R_processed.fif', 'Sujeto26_MF_processed.fif'),
#     ('Sujeto27_R_processed.fif', 'Sujeto27_MF_processed.fif'),
#     ('Sujeto28_R_processed.fif', 'Sujeto28_MF_processed.fif'),
#     ('Sujeto29_R_processed.fif', 'Sujeto29_MF_processed.fif'),
#     ('Sujeto30_R_processed.fif', 'Sujeto30_MF_processed.fif'),
#     ('Sujeto31_R_processed.fif', 'Sujeto31_MF_processed.fif'),
#     ('Sujeto32_R_processed.fif', 'Sujeto32_MF_processed.fif'),
#     ('Sujeto33_R_processed.fif', 'Sujeto33_MF_processed.fif'),
#     ('Sujeto34_R_processed.fif', 'Sujeto34_MF_processed.fif'),
#     #('Sujeto35_R_processed.fif', 'Sujeto35_MF_processed.fif'),
#     ('Sujeto36_R_processed.fif', 'Sujeto36_MF_processed.fif'),
#     ('Sujeto37_R_processed.fif', 'Sujeto37_MF_processed.fif')
# ]

# topoplots_with_zscore_and_significance(file_pairs)


Reading c:\Users\afcad\Downloads\Sujeto1_R_processed.fif ...


C:\Users\afcad\AppData\Local\Temp\ipykernel_1044\2732955412.py:24: RuntimeWarning: This filename (Sujeto1_R_processed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  info = mne.read_epochs(file_pairs[0][0]).info


    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available
Not setting metadata
68 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\afcad\Downloads\Sujeto1_R_processed.fif ...


C:\Users\afcad\AppData\Local\Temp\ipykernel_1044\2732955412.py:32: RuntimeWarning: This filename (Sujeto1_R_processed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs_R = mne.read_epochs(file_R)


    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available
Not setting metadata
68 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\afcad\Downloads\Sujeto1_MF_processed.fif ...


...

    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available
Not setting metadata
133 matching events found
No baseline correction applied
0 projection items activated
Effective window size : 0.250 (s)
Effective window size : 0.250 (s)
Reading c:\Users\afcad\Downloads\Sujeto37_R_processed.fif ...
    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available


C:\Users\afcad\AppData\Local\Temp\ipykernel_1044\2732955412.py:32: RuntimeWarning: This filename (Sujeto37_R_processed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs_R = mne.read_epochs(file_R)


Not setting metadata
71 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\afcad\Downloads\Sujeto37_MF_processed.fif ...


C:\Users\afcad\AppData\Local\Temp\ipykernel_1044\2732955412.py:33: RuntimeWarning: This filename (Sujeto37_MF_processed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs_MF = mne.read_epochs(file_MF)


    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available
Not setting metadata
141 matching events found
No baseline correction applied
0 projection items activated
Effective window size : 0.250 (s)
Effective window size : 0.250 (s)


## Flattenizado

In [ ]:
import mne
import numpy as np
import matplotlib.pyplot as plt

# Extract data
dataR = fileR.get_data()  # Shape: (n_epochs, n_channels, n_times)
dataMF = fileMF.get_data()  # Shape: (n_epochs, n_channels, n_times)

# Sampling frequency
sfreq = fileR.info['sfreq']  # Sampling frequency (Hz)
n_times_to_keep = int(dataR.shape[2] - 0.5 * sfreq)  # Time points to keep

# Trim the last 0.5 seconds
dataR_trimmed = dataR[:, :, :n_times_to_keep]
dataMF_trimmed = dataMF[:, :, :n_times_to_keep]

# print(dataR[:, 1, :])

# Compute mean and standard deviation of the trimmed resting signal
mean_R = np.mean(dataR_trimmed, axis=(0, 2))  # Mean across epochs and time
std_R = np.std(dataR_trimmed, axis=(0, 2))    # Std across epochs and time

# Concatenate both signals along epochs
data_combined = np.concatenate([dataR_trimmed, dataMF_trimmed], axis=0)

# Normalize the concatenated signal
#data_normalized = (data_combined)
data_normalized = (data_combined - mean_R[None, :, None]) / std_R[None, :, None] #Z score

# Concatenate all epochs along the time axis for each channel
data_normalized_ch = np.hstack(data_normalized)

# print(np.shape(data_normalized_ch))

# Assuming data_normalized_ch has shape (n_channels, total_time)
n_channels, total_time = data_normalized_ch.shape
time = np.linspace(0, total_time / sfreq, total_time)  # Generate time axis based on sampling frequency
electrode_names = fileR.ch_names  # List of electrode names

concat_point = dataR_trimmed.shape[0] * dataR_trimmed.shape[2] / sfreq  # Total time of dataR_trimmed
# Create the heatmap
plt.figure(figsize=(12, 6))
plt.imshow(data_normalized_ch, aspect='auto', cmap='viridis', 
           extent=[time[0], time[-1], n_channels, 0],vmin=-1,vmax=3)  # - 1e-5 , 1e-5

# Add a vertical line at the concatenation point
plt.axvline(x=concat_point, color='black', linestyle='--', label='Concatenation Point')

# Add labels and colorbar
plt.colorbar(label='Z-score')
plt.xlabel('Time (s)')
plt.ylabel('Electrodes')
plt.title('Heatmap of Normalized Signal Amplitudes')
plt.yticks(ticks=np.arange(len(electrode_names)), labels=electrode_names)
plt.show()

In [ ]:
# Slice the data to include only the first 5 electrodes
data_subset = data_normalized_ch[:5, :]  # Shape: (5, total_time)

# Create a figure with subplots for heatmap and signals
fig, axes = plt.subplots(6, 1, figsize=(12, 12), gridspec_kw={'height_ratios': [5, 1, 1, 1, 1, 1]}, sharex=True)

# Plot the heatmap in the first subplot
im = axes[0].imshow(data_subset, aspect='auto', cmap='viridis',
                    extent=[time[0], time[-1], 5, 0],vmin=-1,vmax=3)
# Add a vertical line at the concatenation point
axes[0].axvline(x=concat_point, color='black', linestyle='--', label='Concatenation Point')

axes[0].set_ylabel('Electrodes')
axes[0].set_title('Heatmap of Normalized Signal Amplitudes (First 5 Electrodes)')
axes[0].yaxis.set_ticks(range(5))
axes[0].set_yticklabels([f'Electrode {i+1}' for i in range(5)])
plt.colorbar(im, ax=axes[0], orientation='horizontal', fraction=0.05, pad=0.1, label='Amplitude (normalized)')

# Plot signals in the next 5 subplots (one for each electrode)
for i in range(5):
    axes[i + 1].plot(time, data_subset[i], label=f'Electrode {i+1}')
    axes[i + 1].set_ylabel('Amplitude')
    axes[i + 1].legend(loc='upper right')

# Set the x-axis label for the last subplot
axes[-1].set_xlabel('Time (s)')

plt.tight_layout()
plt.show()


## For several Files signal vs time

In [ ]:
import mne
import numpy as np
import matplotlib.pyplot as plt

def process_and_visualize_eeg(file_R, file_MF):
    """
    Process two EEG files (resting and MF signals), normalize them, and create a heatmap.
    
    Parameters:
        file_R (str): Filepath to the resting signal (.fif).
        file_MF (str): Filepath to the MF signal (.fif).
        
    Returns:
        np.ndarray: Normalized concatenated matrix of shape (n_channels, total_time).
    """
    # Load files
    fileR = mne.read_epochs(file_R)  
    fileMF = mne.read_epochs(file_MF)

    # Extract data
    dataR = fileR.get_data()  # Shape: (n_epochs, n_channels, n_times)
    dataMF = fileMF.get_data()  # Shape: (n_epochs, n_channels, n_times)
    
    # Sampling frequency
    sfreq = fileR.info['sfreq']  # Sampling frequency (Hz)
    n_times_to_keep = int(dataR.shape[2] - 0.5 * sfreq)  # Time points to keep

    # Trim the last 0.5 seconds
    dataR_trimmed = dataR[:, :, :n_times_to_keep]
    dataMF_trimmed = dataMF[:, :, :n_times_to_keep]

    # Compute mean and standard deviation of the trimmed resting signal
    mean_R = np.mean(dataR_trimmed, axis=(0, 2))  # Mean across epochs and time
    std_R = np.std(dataR_trimmed, axis=(0, 2))    # Std across epochs and time

    # Concatenate both signals along epochs
    data_combined = np.concatenate([dataR_trimmed, dataMF_trimmed], axis=0)

    # Normalize the concatenated signal
    data_normalized = (data_combined - mean_R[None, :, None]) / std_R[None, :, None]

    # Concatenate all epochs along the time axis for each channel
    data_normalized_ch = np.hstack(data_normalized)

    # Visualization
    n_channels, total_time = data_normalized_ch.shape
    time = np.linspace(0, total_time / sfreq, total_time)  # Generate time axis based on sampling frequency
    electrode_names = fileR.ch_names  # List of electrode names

    # Compute concatenation point in seconds
    concat_point = dataR_trimmed.shape[0] * dataR_trimmed.shape[2] / sfreq  # Total time of dataR_trimmed

    # # Create the heatmap
    # plt.figure(figsize=(12, 6))
    # plt.imshow(data_normalized_ch, aspect='auto', cmap='viridis', 
    #            extent=[time[0], time[-1], n_channels, 0],vmin=-1,vmax=3)  # Extent flips the y-axis to show the first channel at the top

    # # Add a vertical line at the concatenation point
    # plt.axvline(x=concat_point, color='black', linestyle='--', label='Concatenation Point')

    # # Add labels and colorbar
    # plt.colorbar(label='Z-score')
    # plt.xlabel('Time (s)')
    # plt.ylabel('Electrodes')
    # plt.title('Heatmap of Normalized Signal Amplitudes')
    # plt.yticks(ticks=np.arange(len(electrode_names)), labels=electrode_names)
    # plt.legend()
    # plt.show()

    return data_normalized_ch

file_pairs = [
    ('Sujeto4_R_processed.fif', 'Sujeto4_MF_processed.fif'),
    ('Sujeto5_R_processed.fif', 'Sujeto5_MF_processed.fif'),
    ('Sujeto6_R_processed.fif', 'Sujeto6_MF_processed.fif'),
    ('Sujeto7_R_processed.fif', 'Sujeto7_MF_processed.fif'),
    ('Sujeto8_R_processed.fif', 'Sujeto8_MF_processed.fif'),
    ('Sujeto9_R_processed.fif', 'Sujeto9_MF_processed.fif'),
    ('Sujeto10_R_processed.fif', 'Sujeto10_MF_processed.fif'),
    ('Sujeto11_R_processed.fif', 'Sujeto11_MF_processed.fif'),
    ('Sujeto12_R_processed.fif', 'Sujeto12_MF_processed.fif'),
    ('Sujeto13_R_processed.fif', 'Sujeto13_MF_processed.fif'),
    ('Sujeto14_R_processed.fif', 'Sujeto14_MF_processed.fif'),
    ('Sujeto15_R_processed.fif', 'Sujeto15_MF_processed.fif'),
    ('Sujeto16_R_processed.fif', 'Sujeto16_MF_processed.fif'),
    ('Sujeto18_R_processed.fif', 'Sujeto18_MF_processed.fif'),
    ('Sujeto19_R_processed.fif', 'Sujeto19_MF_processed.fif'),
    ('Sujeto20_R_processed.fif', 'Sujeto20_MF_processed.fif'),
    ('Sujeto21_R_processed.fif', 'Sujeto21_MF_processed.fif'),
    ('Sujeto22_R_processed.fif', 'Sujeto22_MF_processed.fif'),
    ('Sujeto23_R_processed.fif', 'Sujeto23_MF_processed.fif'),
    ('Sujeto24_R_processed.fif', 'Sujeto24_MF_processed.fif'),
    ('Sujeto25_R_processed.fif', 'Sujeto25_MF_processed.fif'),
    ('Sujeto26_R_processed.fif', 'Sujeto26_MF_processed.fif'),
    ('Sujeto27_R_processed.fif', 'Sujeto27_MF_processed.fif'),
    ('Sujeto28_R_processed.fif', 'Sujeto28_MF_processed.fif'),
    ('Sujeto29_R_processed.fif', 'Sujeto29_MF_processed.fif')
]

# Initialize list to store the matrices
normalized_matrices = []

# Process each pair of files
for file_R, file_MF in file_pairs:
    normalized_matrix = process_and_visualize_eeg(file_R, file_MF)
    normalized_matrices.append(normalized_matrix)

# Now calculate the average matrix
# To ensure that all matrices are the same shape, we trim them to the smallest one
min_shape = min([matrix.shape[1] for matrix in normalized_matrices])  # Smallest time dimension
normalized_matrices_trimmed = [matrix[:, :min_shape] for matrix in normalized_matrices]

# Calculate the average across all matrices
average_matrix = np.mean(np.array(normalized_matrices_trimmed), axis=0)

# Create the heatmap for the average matrix
plt.figure(figsize=(12, 6))
plt.imshow(average_matrix, aspect='auto', cmap='viridis', 
            vmin=-1, vmax=1)  # Extent flips the y-axis to show the first channel at the top
plt.colorbar(label='Z-score')
plt.xlabel('Time (s)')
plt.ylabel('Electrodes')
plt.title('Heatmap of Average Normalized Signal Amplitudes')
plt.show()

# For several files PSD

In [ ]:
import mne
import numpy as np
import matplotlib.pyplot as plt

def calculate_average_psd(file_pairs, fmin, fmax):
    all_psd_R = []
    all_psd_MF = []
    freqs = None
    electrode_names = None

    for file_R, file_MF in file_pairs:
        # Load files
        fileR = mne.read_epochs(file_R)
        fileMF = mne.read_epochs(file_MF)

        # Get electrode names (only once)
        if electrode_names is None:
            electrode_names = fileR.ch_names

        # Compute PSD
        psd_R = fileR.compute_psd(method='welch', fmin=fmin, fmax=fmax)
        psd_MF = fileMF.compute_psd(method='welch', fmin=fmin, fmax=fmax)

        # Get PSD values and frequencies
        psd_values_R, freqs = psd_R.get_data(return_freqs=True)
        psd_values_MF, _ = psd_MF.get_data(return_freqs=True)

        # Append PSD values to the list
        all_psd_R.append(psd_values_R)
        all_psd_MF.append(psd_values_MF)

    # Calculate average PSD across all files
    avg_psd_R = np.mean(np.concatenate(all_psd_R, axis=0), axis=0)  # Shape: (61, 99)
    avg_psd_MF = np.mean(np.concatenate(all_psd_MF, axis=0), axis=0)  # Shape: (61, 99)

    # Plot the results
    fig, axs = plt.subplots(3, 1, figsize=(10, 12))

    # Plot for average psd_R (all channels)
    for channel, name in enumerate(electrode_names):
        axs[0].plot(freqs, 10 * np.log10(avg_psd_R[channel] / 1e-12), label=name)
    axs[0].set_xlabel("Frequency (Hz)")
    axs[0].set_ylabel("PSD (dB)")
    axs[0].set_title("Average PSD Resting")

    # Plot for average psd_MF (all channels)
    for channel, name in enumerate(electrode_names):
        axs[1].plot(freqs, 10 * np.log10(avg_psd_MF[channel] / 1e-12), label=name)
    axs[1].set_xlabel("Frequency (Hz)")
    axs[1].set_ylabel("PSD (dB)")
    axs[1].set_title("Average PSD Mindfulness")

    # Calculate the difference (MF - R) and plot it (all channels)
    for channel, name in enumerate(electrode_names):
        difference = 10 * np.log10(avg_psd_MF[channel] / 1e-12) - 10 * np.log10(avg_psd_R[channel] / 1e-12)
        axs[2].plot(freqs, difference, label=name)
    axs[2].set_xlabel("Frequency (Hz)")
    axs[2].set_ylabel("Difference (dB)")
    axs[2].set_title("Average Difference (MF - R)")

    # Create a single legend for all subplots
    handles, labels = axs[0].get_legend_handles_labels()
    fig.legend(handles, labels, fontsize='small')

    plt.tight_layout()
    plt.show()

file_pairs = [
    ('Sujeto1_R_processed.fif', 'Sujeto1_MF_processed.fif'),
    ('Sujeto2_R_processed.fif', 'Sujeto2_MF_processed.fif'),
    ('Sujeto3_R_processed.fif', 'Sujeto3_MF_processed.fif'),
    ('Sujeto4_R_processed.fif', 'Sujeto4_MF_processed.fif'),
    ('Sujeto5_R_processed.fif', 'Sujeto5_MF_processed.fif'),
    ('Sujeto6_R_processed.fif', 'Sujeto6_MF_processed.fif'),
    ('Sujeto7_R_processed.fif', 'Sujeto7_MF_processed.fif'),
    ('Sujeto8_R_processed.fif', 'Sujeto8_MF_processed.fif'),
    ('Sujeto9_R_processed.fif', 'Sujeto9_MF_processed.fif'),
    ('Sujeto10_R_processed.fif', 'Sujeto10_MF_processed.fif'),
    ('Sujeto11_R_processed.fif', 'Sujeto11_MF_processed.fif'),
    # ('Sujeto12_R_processed.fif', 'Sujeto12_MF_processed.fif'),
    ('Sujeto13_R_processed.fif', 'Sujeto13_MF_processed.fif'),
    ('Sujeto14_R_processed.fif', 'Sujeto14_MF_processed.fif'),
    ('Sujeto15_R_processed.fif', 'Sujeto15_MF_processed.fif'),
    ('Sujeto16_R_processed.fif', 'Sujeto16_MF_processed.fif'),
    ('Sujeto17_R_processed.fif', 'Sujeto17_MF_processed.fif'),
    ('Sujeto18_R_processed.fif', 'Sujeto18_MF_processed.fif'),
    ('Sujeto19_R_processed.fif', 'Sujeto19_MF_processed.fif'),
    ('Sujeto20_R_processed.fif', 'Sujeto20_MF_processed.fif'),
    ('Sujeto21_R_processed.fif', 'Sujeto21_MF_processed.fif'),
    ('Sujeto22_R_processed.fif', 'Sujeto22_MF_processed.fif'),
    ('Sujeto23_R_processed.fif', 'Sujeto23_MF_processed.fif'),
    ('Sujeto24_R_processed.fif', 'Sujeto24_MF_processed.fif'),
    ('Sujeto25_R_processed.fif', 'Sujeto25_MF_processed.fif'),
    ('Sujeto26_R_processed.fif', 'Sujeto26_MF_processed.fif'),
    ('Sujeto27_R_processed.fif', 'Sujeto27_MF_processed.fif'),
    ('Sujeto28_R_processed.fif', 'Sujeto28_MF_processed.fif'),
    ('Sujeto29_R_processed.fif', 'Sujeto29_MF_processed.fif'),
    ('Sujeto30_R_processed.fif', 'Sujeto30_MF_processed.fif'),
    ('Sujeto31_R_processed.fif', 'Sujeto31_MF_processed.fif'),
    ('Sujeto32_R_processed.fif', 'Sujeto32_MF_processed.fif'),
    ('Sujeto33_R_processed.fif', 'Sujeto33_MF_processed.fif'),
    ('Sujeto34_R_processed.fif', 'Sujeto34_MF_processed.fif'),
    #('Sujeto35_R_processed.fif', 'Sujeto35_MF_processed.fif'),
    ('Sujeto36_R_processed.fif', 'Sujeto36_MF_processed.fif'),
    ('Sujeto37_R_processed.fif', 'Sujeto37_MF_processed.fif')
]

fmin = 0.5  # Minimum frequency
fmax = 50  # Maximum frequency

# Run the function
calculate_average_psd(file_pairs, fmin, fmax)


In [ ]:
import random

file_pairs = [
    ('Sujeto1_R_processed.fif', 'Sujeto1_MF_processed.fif'),
    ('Sujeto2_R_processed.fif', 'Sujeto2_MF_processed.fif'),
    ('Sujeto3_R_processed.fif', 'Sujeto3_MF_processed.fif'),
    ('Sujeto4_R_processed.fif', 'Sujeto4_MF_processed.fif'),
    ('Sujeto5_R_processed.fif', 'Sujeto5_MF_processed.fif'),
    ('Sujeto6_R_processed.fif', 'Sujeto6_MF_processed.fif'),
    ('Sujeto7_R_processed.fif', 'Sujeto7_MF_processed.fif'),
    ('Sujeto8_R_processed.fif', 'Sujeto8_MF_processed.fif'),
    ('Sujeto9_R_processed.fif', 'Sujeto9_MF_processed.fif'),
    ('Sujeto10_R_processed.fif', 'Sujeto10_MF_processed.fif'),
    ('Sujeto12_R_processed.fif', 'Sujeto12_MF_processed.fif'),
    ('Sujeto11_R_processed.fif', 'Sujeto11_MF_processed.fif'),
    # ('Sujeto12_R_processed.fif', 'Sujeto12_MF_processed.fif'),
    ('Sujeto13_R_processed.fif', 'Sujeto13_MF_processed.fif'),
    ('Sujeto14_R_processed.fif', 'Sujeto14_MF_processed.fif'),
    ('Sujeto15_R_processed.fif', 'Sujeto15_MF_processed.fif'),
    ('Sujeto16_R_processed.fif', 'Sujeto16_MF_processed.fif'),
    ('Sujeto17_R_processed.fif', 'Sujeto17_MF_processed.fif'),
    ('Sujeto18_R_processed.fif', 'Sujeto18_MF_processed.fif'),
    ('Sujeto19_R_processed.fif', 'Sujeto19_MF_processed.fif'),
    ('Sujeto20_R_processed.fif', 'Sujeto20_MF_processed.fif'),
    ('Sujeto21_R_processed.fif', 'Sujeto21_MF_processed.fif'),
    ('Sujeto22_R_processed.fif', 'Sujeto22_MF_processed.fif'),
    ('Sujeto23_R_processed.fif', 'Sujeto23_MF_processed.fif'),
    ('Sujeto24_R_processed.fif', 'Sujeto24_MF_processed.fif'),
    ('Sujeto25_R_processed.fif', 'Sujeto25_MF_processed.fif'),
    ('Sujeto26_R_processed.fif', 'Sujeto26_MF_processed.fif'),
    ('Sujeto27_R_processed.fif', 'Sujeto27_MF_processed.fif'),
    ('Sujeto28_R_processed.fif', 'Sujeto28_MF_processed.fif'),
    ('Sujeto29_R_processed.fif', 'Sujeto29_MF_processed.fif'),
    ('Sujeto30_R_processed.fif', 'Sujeto30_MF_processed.fif'),
    ('Sujeto31_R_processed.fif', 'Sujeto31_MF_processed.fif'),
    ('Sujeto32_R_processed.fif', 'Sujeto32_MF_processed.fif'),
    ('Sujeto33_R_processed.fif', 'Sujeto33_MF_processed.fif'),
    ('Sujeto34_R_processed.fif', 'Sujeto34_MF_processed.fif'),
    #('Sujeto35_R_processed.fif', 'Sujeto35_MF_processed.fif'),
    ('Sujeto36_R_processed.fif', 'Sujeto36_MF_processed.fif'),
    ('Sujeto37_R_processed.fif', 'Sujeto37_MF_processed.fif')
]

# Generate 10 random combinations of 5 pairs
random_combinations = [random.sample(file_pairs, 10) for _ in range(10)]

fmin=0.5
fmax=50

# Print the combinations
for i, combination in enumerate(random_combinations, 1):
    print(f"Combination {i}: {combination}")
    calculate_average_psd(file_pairs, fmin, fmax)

#### Correlation Data- Questionary...

In [ ]:
import numpy as np
import mne
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

file_pairs = [
    ('Sujeto1_R_processed.fif', 'Sujeto1_MF_processed.fif'),
    ('Sujeto2_R_processed.fif', 'Sujeto2_MF_processed.fif'),
    ('Sujeto3_R_processed.fif', 'Sujeto3_MF_processed.fif')
]

# Questionnaire responses (adjust based on participant order)
responses = np.array([4, 4, 6])  

# Extract EEG power differences from `topoplotsAverage`
relative_power_difference_all = []  

for str_FileName_R, str_FileName_MF in file_pairs:
    fileR = mne.read_epochs(str_FileName_R)  
    fileMF = mne.read_epochs(str_FileName_MF)

    psd_R, _ = fileR.compute_psd(method='welch', fmin=8, fmax=12).get_data(return_freqs=True)
    psd_MF, _ = fileMF.compute_psd(method='welch', fmin=8, fmax=12).get_data(return_freqs=True)
    
    # Average across epochs and frequencies
    relative_psd_R = np.mean(psd_R, axis=(0, 2))  
    relative_psd_MF = np.mean(psd_MF, axis=(0, 2))  

    relative_power_difference_all.append(relative_psd_MF - relative_psd_R)  

relative_power_difference_all = np.array(relative_power_difference_all)  # Shape: (Participants, Electrodes)

# Compute correlation per electrode
correlations = np.array([pearsonr(relative_power_difference_all[:, i], responses)[0] for i in range(relative_power_difference_all.shape[1])])

# Load electrode locations
info = mne.read_epochs(file_pairs[0][0]).info  

fig, ax = plt.subplots(figsize=(6, 5))

# Generate the topomap
im, _ = mne.viz.plot_topomap(correlations, info, axes=ax, cmap='coolwarm', show=False)

# Add colorbar using the image object from the topomap
cbar = plt.colorbar(im, ax=ax, shrink=0.6, label='Correlation (r)')
cbar.ax.tick_params(labelsize=10)  # Adjust tick labels for readability

plt.title('Correlation Between EEG and Questionnaire Scores')
plt.show()


In [ ]:
import pandas as pd
import numpy as np

# df = pd.read_excel('Analisis conductuales - Datos conductuales.xlsx')
df = pd.read_excel('EEG-Datos conductuales.xlsx')
#print(df.head())

sujeto_column = df['Sujeto']
data_column = df['MWQT'] #Auto-regulación Confianza Valencia positiva Valencia Negativa

print(f"Initial counts - Sujetos: {len(sujeto_column)}, Responses: {len(data_column)}")

# Create a mask where both columns are not NaN
mask = sujeto_column.notna() & data_column.notna()

eliminate_mask = sujeto_column.isna() | data_column.isna()
# Get the Sujeto values that will be removed
eliminated_sujetos = sujeto_column[eliminate_mask]

print("\nEliminated due to NaN values:")
print(eliminated_sujetos.to_string())  # to_string() shows all values

# Apply the initial mask to both columns
sujeto_clean = sujeto_column[mask]
data_clean = data_column[mask]

# Now exclude specific subjects
subjects_to_exclude = ['SUJ_12', 'SUJ_35']  # Add more as needed
additional_elim_mask = sujeto_clean.isin(subjects_to_exclude)

print(f"\nAdditional subjects being excluded: {subjects_to_exclude}")
print("These subjects will be removed:")
print(sujeto_clean[additional_elim_mask].to_string())

# Final clean mask
final_mask = ~additional_elim_mask
sujeto_final = sujeto_clean[final_mask]
data_final = data_clean[final_mask]

print(f"\nFinal counts - Sujetos: {len(sujeto_final)}, Responses: {len(data_final)}")
print("\nCleaned Sujetos:")
print(sujeto_final.to_string())
print("\nCleaned Data (first 10 values):")
print(data_final.head(10).to_string())

# Convert to numpy array
responses_array = data_final.to_numpy()
print(f"\nNumPy array shape: {responses_array.shape}")
print(f"First 5 array values: {responses_array[:5]}")

### Questionary for all files in all band frequencies

In [ ]:
import numpy as np
import mne
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

# Define frequency bands
frequency_bands = {
    'Delta (0.5-4 Hz)': (0.5, 4),
    'Theta (4-8 Hz)': (4, 8),
    'Alpha (8-12 Hz)': (8, 12),
    'Beta (12-18 Hz)': (12, 18),
    'Fast Beta (18-30 Hz)': (18, 30)
}

file_pairs = [
    ('Sujeto1_R_processed.fif', 'Sujeto1_MF_processed.fif'),
    ('Sujeto2_R_processed.fif', 'Sujeto2_MF_processed.fif'),
    ('Sujeto3_R_processed.fif', 'Sujeto3_MF_processed.fif'),
    ('Sujeto4_R_processed.fif', 'Sujeto4_MF_processed.fif'),
    ('Sujeto5_R_processed.fif', 'Sujeto5_MF_processed.fif'),
    ('Sujeto6_R_processed.fif', 'Sujeto6_MF_processed.fif'),
    ('Sujeto7_R_processed.fif', 'Sujeto7_MF_processed.fif'),
    ('Sujeto8_R_processed.fif', 'Sujeto8_MF_processed.fif'),
    ('Sujeto9_R_processed.fif', 'Sujeto9_MF_processed.fif'),
    ('Sujeto10_R_processed.fif', 'Sujeto10_MF_processed.fif'),
    ('Sujeto11_R_processed.fif', 'Sujeto11_MF_processed.fif'),
    #('Sujeto12_R_processed.fif', 'Sujeto12_MF_processed.fif'), #always excluded
    ('Sujeto13_R_processed.fif', 'Sujeto13_MF_processed.fif'),
    ('Sujeto14_R_processed.fif', 'Sujeto14_MF_processed.fif'),
    ('Sujeto15_R_processed.fif', 'Sujeto15_MF_processed.fif'),
    ('Sujeto16_R_processed.fif', 'Sujeto16_MF_processed.fif'),
    ('Sujeto17_R_processed.fif', 'Sujeto17_MF_processed.fif'),
    ('Sujeto18_R_processed.fif', 'Sujeto18_MF_processed.fif'),
    ('Sujeto19_R_processed.fif', 'Sujeto19_MF_processed.fif'),
    ('Sujeto20_R_processed.fif', 'Sujeto20_MF_processed.fif'),
    ('Sujeto21_R_processed.fif', 'Sujeto21_MF_processed.fif'),
    ('Sujeto22_R_processed.fif', 'Sujeto22_MF_processed.fif'),
    ('Sujeto23_R_processed.fif', 'Sujeto23_MF_processed.fif'),
    ('Sujeto24_R_processed.fif', 'Sujeto24_MF_processed.fif'),
    ('Sujeto25_R_processed.fif', 'Sujeto25_MF_processed.fif'),
    ('Sujeto26_R_processed.fif', 'Sujeto26_MF_processed.fif'),
    ('Sujeto27_R_processed.fif', 'Sujeto27_MF_processed.fif'),
    ('Sujeto28_R_processed.fif', 'Sujeto28_MF_processed.fif'),
    ('Sujeto29_R_processed.fif', 'Sujeto29_MF_processed.fif'),
    ('Sujeto30_R_processed.fif', 'Sujeto30_MF_processed.fif'),
    ('Sujeto31_R_processed.fif', 'Sujeto31_MF_processed.fif'),
    ('Sujeto32_R_processed.fif', 'Sujeto32_MF_processed.fif'),
    ('Sujeto33_R_processed.fif', 'Sujeto33_MF_processed.fif'),
    ('Sujeto34_R_processed.fif', 'Sujeto34_MF_processed.fif'),
    #('Sujeto35_R_processed.fif', 'Sujeto35_MF_processed.fif'),  #always excluded
    ('Sujeto36_R_processed.fif', 'Sujeto36_MF_processed.fif'),
    ('Sujeto37_R_processed.fif', 'Sujeto37_MF_processed.fif')
]

# Questionnaire responses
#responses = np.array([4, 4, 6, 7, 7, 1, 8, 7, 9, 2, 5, 7, 4, 6, 8, 6, 7, 7, 6, 8, 8, 4, 8, 9, 7, 8, 5, 7, 5, 4, 7, 4, 7, 6])  #AR2-MF
#responses = np.array([0, 3, 6, 1, 2, 4, 5, 6, 7, 1, 5, 8, 3, 1, 7, 4, 7, 8, 4, 4, 5, 1, 5, 2, 5, 6, 4, 1, 2, 1, 4, 0, 8, 1])  #AR3-MF
# responses = np.array([2, 5, 5, 5, 5, 3, 5, 6, 7, 1, 6, 4, 2, 7, 7, 3, 6, 4, 7, 5, 6, 3, 8, 9, 6, 5, 7, 5, 4, 3, 5, 1, 7, 6])  #AR4-MF
responses = np.array(responses_array.tolist())

# Load electrode locations (use first file)
info = mne.read_epochs(file_pairs[0][0]).info  

# Create figure with subplots
fig, axes = plt.subplots(1, len(frequency_bands), figsize=(15, 4))

for ax, (band_name, (fmin, fmax)) in zip(axes, frequency_bands.items()):
    relative_power_difference_all = []

    # Compute PSD differences for each subject
    for str_FileName_R, str_FileName_MF in file_pairs:
        fileR = mne.read_epochs(str_FileName_R)  
        fileMF = mne.read_epochs(str_FileName_MF)

        psd_R, _ = fileR.compute_psd(method='welch', fmin=fmin, fmax=fmax).get_data(return_freqs=True)
        psd_MF, _ = fileMF.compute_psd(method='welch', fmin=fmin, fmax=fmax).get_data(return_freqs=True)

        avg_psd_R = np.mean(psd_R, axis=(0, 2))  # Mean across epochs
        avg_psd_MF = np.mean(psd_MF, axis=(0, 2))  
        relative_power_difference_all.append(avg_psd_MF - avg_psd_R)  

    relative_power_difference_all = np.array(relative_power_difference_all)  # Shape: (Participants, Electrodes)

    # Compute correlation per electrode
    correlations = np.array([
        pearsonr(relative_power_difference_all[:, i], responses)[0] 
        for i in range(relative_power_difference_all.shape[1])
    ])

    # Plot topomap
    im, _ = mne.viz.plot_topomap(correlations, info, axes=ax, cmap='jet', show=False)
    
    ax.set_title(band_name, fontsize=10)

# Adjust layout and add colorbar
cbar = plt.colorbar(im, ax=axes.ravel().tolist(), shrink=0.6, label='Correlation (r)')
plt.suptitle('Correlation Between EEG Power and Questionnaire Scores', fontsize=12)
plt.tight_layout()
plt.show()

# Questionaries with significance

In [ ]:
import numpy as np
import mne
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

def topomap_correlation_with_questionnaire(file_pairs, responses, frequency_bands, title='Correlation with Questionnaire'):
    """
    Plot EEG power–questionnaire correlation per frequency band:
    - Pearson r
    - p-values
    - Significant r (p < 0.05)

    Parameters:
    - file_pairs: list of (R, MF) file tuples
    - responses: array of questionnaire responses
    - frequency_bands: dict of frequency band names to (fmin, fmax)
    - title: plot title
    """
    info = mne.read_epochs(file_pairs[0][0]).info
    n_bands = len(frequency_bands)
    n_channels = len(info['ch_names'])

    fig, axes = plt.subplots(3, n_bands, figsize=(3.5 * n_bands, 9))

    for col, (band_name, (fmin, fmax)) in enumerate(frequency_bands.items()):
        rel_power_diff_all = []

        # Collect power differences for all subjects
        for file_R, file_MF in file_pairs:
            epochs_R = mne.read_epochs(file_R)
            epochs_MF = mne.read_epochs(file_MF)

            psd_R, freqs = epochs_R.compute_psd(method='welch', fmin=fmin, fmax=fmax).get_data(return_freqs=True)
            psd_MF, _ = epochs_MF.compute_psd(method='welch', fmin=fmin, fmax=fmax).get_data(return_freqs=True)

            avg_psd_R = np.mean(psd_R, axis=(0, 2))  # (n_channels,)
            avg_psd_MF = np.mean(psd_MF, axis=(0, 2))

            rel_power_diff_all.append(avg_psd_MF - avg_psd_R)

        rel_power_diff_all = np.array(rel_power_diff_all)  # shape (n_subjects, n_channels)

        # Compute Pearson r and p-values
        r_vals = []
        p_vals = []
        for ch in range(n_channels):
            r, p = pearsonr(rel_power_diff_all[:, ch], responses)
            r_vals.append(r)
            p_vals.append(p)
        r_vals = np.array(r_vals)
        p_vals = np.array(p_vals)
        sig_mask = p_vals < 0.05
        r_sig = np.where(sig_mask, r_vals, 0)  # Use 0 instead of NaN

        # Plotting
        # First row: Pearson r values
        im, _ = mne.viz.plot_topomap(r_vals, info, axes=axes[0, col], cmap='coolwarm', show=False)
        axes[0, col].set_title(f'{band_name}\nr', fontsize=10)
        plt.colorbar(im, ax=axes[0, col], orientation='horizontal', shrink=0.6)

        # Second row: p-values
        im, _ = mne.viz.plot_topomap(p_vals, info, axes=axes[1, col], cmap='magma_r', show=False)
        axes[1, col].set_title('p-values', fontsize=10)
        plt.colorbar(im, ax=axes[1, col], orientation='horizontal', shrink=0.6)

        # Third row: Significant correlations only
        im = mne.viz.plot_topomap(
            p_vals, 
            info, 
            axes=axes[2, col], 
            cmap='jet', 
            show=False,
            #mask=sig_mask.reshape(-1, 1),
            #mask_params=dict(markersize=0),
            vlim=(0,0.1)
        )[0]  # Get just the image part of the tuple
        axes[2, col].set_title('r (p<0.05)', fontsize=10)
        plt.colorbar(im, ax=axes[2, col], orientation='horizontal', shrink=0.6)

    # Add row labels
    for ax, label in zip(axes[:, 0], ['Pearson r', 'p-value', 'Significant r']):
        ax.annotate(label, xy=(-0.2, 0.5), xycoords='axes fraction', va='center', ha='right', rotation=90, fontsize=12)

    plt.suptitle(title, fontsize=14)
    plt.tight_layout()
    plt.show()

# Your frequency bands
frequency_bands = {
    'Delta (0.5–4 Hz)': (0.5, 4),
    'Theta (4–8 Hz)': (4, 8),
    'Alpha (8–12 Hz)': (8, 12),
    'Beta (12–18 Hz)': (12, 18),
    'Fast Beta (18–30 Hz)': (18, 30)
}

file_pairs = [
    ('Sujeto1_R_processed.fif', 'Sujeto1_MF_processed.fif'),
    ('Sujeto2_R_processed.fif', 'Sujeto2_MF_processed.fif'),
    ('Sujeto3_R_processed.fif', 'Sujeto3_MF_processed.fif'),
    ('Sujeto4_R_processed.fif', 'Sujeto4_MF_processed.fif'),
    ('Sujeto5_R_processed.fif', 'Sujeto5_MF_processed.fif'),
    ('Sujeto6_R_processed.fif', 'Sujeto6_MF_processed.fif'),
    ('Sujeto7_R_processed.fif', 'Sujeto7_MF_processed.fif'),
    ('Sujeto8_R_processed.fif', 'Sujeto8_MF_processed.fif'),
    ('Sujeto9_R_processed.fif', 'Sujeto9_MF_processed.fif'),
    ('Sujeto10_R_processed.fif', 'Sujeto10_MF_processed.fif'),
    ('Sujeto11_R_processed.fif', 'Sujeto11_MF_processed.fif'),
    #('Sujeto12_R_processed.fif', 'Sujeto12_MF_processed.fif'),  #always excluded
    ('Sujeto13_R_processed.fif', 'Sujeto13_MF_processed.fif'),
    ('Sujeto14_R_processed.fif', 'Sujeto14_MF_processed.fif'),
    ('Sujeto15_R_processed.fif', 'Sujeto15_MF_processed.fif'),
    ('Sujeto16_R_processed.fif', 'Sujeto16_MF_processed.fif'),
    ('Sujeto17_R_processed.fif', 'Sujeto17_MF_processed.fif'),
    ('Sujeto18_R_processed.fif', 'Sujeto18_MF_processed.fif'),
    ('Sujeto19_R_processed.fif', 'Sujeto19_MF_processed.fif'),
    ('Sujeto20_R_processed.fif', 'Sujeto20_MF_processed.fif'),
    ('Sujeto21_R_processed.fif', 'Sujeto21_MF_processed.fif'),
    ('Sujeto22_R_processed.fif', 'Sujeto22_MF_processed.fif'),
    ('Sujeto23_R_processed.fif', 'Sujeto23_MF_processed.fif'),
    ('Sujeto24_R_processed.fif', 'Sujeto24_MF_processed.fif'),
    ('Sujeto25_R_processed.fif', 'Sujeto25_MF_processed.fif'),
    ('Sujeto26_R_processed.fif', 'Sujeto26_MF_processed.fif'),
    ('Sujeto27_R_processed.fif', 'Sujeto27_MF_processed.fif'),
    ('Sujeto28_R_processed.fif', 'Sujeto28_MF_processed.fif'),
    ('Sujeto29_R_processed.fif', 'Sujeto29_MF_processed.fif'),
    ('Sujeto30_R_processed.fif', 'Sujeto30_MF_processed.fif'),
    ('Sujeto31_R_processed.fif', 'Sujeto31_MF_processed.fif'),
    ('Sujeto32_R_processed.fif', 'Sujeto32_MF_processed.fif'),
    ('Sujeto33_R_processed.fif', 'Sujeto33_MF_processed.fif'),
    ('Sujeto34_R_processed.fif', 'Sujeto34_MF_processed.fif'),
    #('Sujeto35_R_processed.fif', 'Sujeto35_MF_processed.fif'), #always excluded
    ('Sujeto36_R_processed.fif', 'Sujeto36_MF_processed.fif'),
    ('Sujeto37_R_processed.fif', 'Sujeto37_MF_processed.fif')
    ]
#responses = np.array([4, 4, 6, 7, 7, 1, 8, 7, 9, 2, 5, 7, 4, 6, 8, 6, 7, 7, 6, 8, 8, 4, 8, 9, 7, 8, 5, 7, 5, 4, 7, 4, 7, 6])  #AR2-MF
#responses = np.array([0, 3, 6, 1, 2, 4, 5, 6, 7, 1, 5, 8, 3, 1, 7, 4, 7, 8, 4, 4, 5, 1, 5, 2, 5, 6, 4, 1, 2, 1, 4, 0, 8, 1])  #AR3-MF
#responses = np.array([2, 5, 5, 5, 5, 3, 5, 6, 7, 1, 6, 4, 2, 7, 7, 3, 6, 4, 7, 5, 6, 3, 8, 9, 6, 5, 7, 5, 4, 3, 5, 1, 7, 6])  #AR4-MF
responses = np.array(responses_array.tolist())

# file_pairs=file_pairs[:5]
# responses=responses[:5]

# Call the function
topomap_correlation_with_questionnaire(file_pairs, responses, frequency_bands, title='MWQ5 Correlation')


# Significance correlation in both states

In [19]:
import numpy as np
import mne
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import pandas as pd


# df = pd.read_excel('Analisis conductuales - Datos conductuales.xlsx')
df = pd.read_excel('EEG-Datos conductuales.xlsx')
#print(df.head())

sujeto_column = df['Sujeto']
data_column = df['MWQT'] #Auto-regulación Confianza Valencia positiva Valencia Negativa

print(f"Initial counts - Sujetos: {len(sujeto_column)}, Responses: {len(data_column)}")

# Create a mask where both columns are not NaN
mask = sujeto_column.notna() & data_column.notna()

eliminate_mask = sujeto_column.isna() | data_column.isna()
# Get the Sujeto values that will be removed
eliminated_sujetos = sujeto_column[eliminate_mask]

print("\nEliminated due to NaN values:")
print(eliminated_sujetos.to_string())  # to_string() shows all values

# Apply the initial mask to both columns
sujeto_clean = sujeto_column[mask]
data_clean = data_column[mask]

# Now exclude specific subjects
subjects_to_exclude = ['SUJ_12', 'SUJ_35']  # Add more as needed
additional_elim_mask = sujeto_clean.isin(subjects_to_exclude)

print(f"\nAdditional subjects being excluded: {subjects_to_exclude}")
print("These subjects will be removed:")
print(sujeto_clean[additional_elim_mask].to_string())

# Final clean mask
final_mask = ~additional_elim_mask
sujeto_final = sujeto_clean[final_mask]
data_final = data_clean[final_mask]

print(f"\nFinal counts - Sujetos: {len(sujeto_final)}, Responses: {len(data_final)}")
print("\nCleaned Sujetos:")
print(sujeto_final.to_string())
print("\nCleaned Data (first 10 values):")
print(data_final.head(10).to_string())

# Convert to numpy array
responses_array = data_final.to_numpy()
print(f"\nNumPy array shape: {responses_array.shape}")
print(f"First 5 array values: {responses_array[:5]}")

def topomap_correlation_states(file_pairs, responses, frequency_bands, title='Correlation with Questionnaire'):

    info = mne.read_epochs(file_pairs[0][0]).info
    n_bands = len(frequency_bands)
    n_channels = len(info['ch_names'])

    fig, axes = plt.subplots(3, n_bands, figsize=(3.5 * n_bands, 10))

    for col, (band_name, (fmin, fmax)) in enumerate(frequency_bands.items()):
        psd_R_all = []
        psd_MF_all = []

        # --- Extract and average PSDs ---
        for file_R, file_MF in file_pairs:
            epochs_R = mne.read_epochs(file_R, preload=True)
            epochs_MF = mne.read_epochs(file_MF, preload=True)

            psd_R, freqs = epochs_R.compute_psd(method='welch', fmin=fmin, fmax=fmax).get_data(return_freqs=True)
            psd_MF, _ = epochs_MF.compute_psd(method='welch', fmin=fmin, fmax=fmax).get_data(return_freqs=True)

            avg_psd_R = np.mean(psd_R, axis=(0, 2))  # shape: (n_channels,)
            avg_psd_MF = np.mean(psd_MF, axis=(0, 2))

            psd_R_all.append(avg_psd_R)
            psd_MF_all.append(avg_psd_MF)

        psd_R_all = np.array(psd_R_all)      # shape: (n_subjects, n_channels)
        psd_MF_all = np.array(psd_MF_all)
        psd_diff = psd_MF_all - psd_R_all

        # --- Correlation for each condition ---
        for row, (state_name, data) in enumerate([
            ("Resting", psd_R_all),
            ("Mindfulness", psd_MF_all),
            ("Difference (MF - R)", psd_diff)
        ]):
            r_vals = np.zeros(n_channels)
            p_vals = np.zeros(n_channels)

            for ch in range(n_channels):
                r_vals[ch], p_vals[ch] = pearsonr(data[:, ch], responses)

            sig_mask = p_vals < 0.05  # Threshold for significance

            print(f'{state_name} - {band_name}: {np.sum(sig_mask)} significant channels (p < 0.05)')

            im, cn = mne.viz.plot_topomap(
                r_vals, info, axes=axes[row, col], cmap='jet', show=False,
                names=info['ch_names'],
                vlim=(-1, 1),
                mask=sig_mask,
                mask_params=dict(marker='*', markersize=25, markerfacecolor='black')
            )
            axes[row, col].set_title(f'{band_name}\n{state_name}', fontsize=10)
            plt.colorbar(im, ax=axes[row, col], orientation='horizontal', shrink=0.6)

    # --- Row Labels ---
    for ax, label in zip(axes[:, 0], ['Resting', 'Mindfulness', 'Difference']):
        ax.annotate(label, xy=(-0.2, 0.5), xycoords='axes fraction',
                    va='center', ha='right', rotation=90, fontsize=12)

    plt.suptitle(title, fontsize=16)
    plt.tight_layout()
    plt.show()


# Your frequency bands
frequency_bands = {
    'Delta (0.5–4 Hz)': (0.5, 4),
    'Theta (4–8 Hz)': (4, 8),
    'Alpha (8–12 Hz)': (8, 12),
    'Beta (12–18 Hz)': (12, 18),
    'Fast Beta (18–30 Hz)': (18, 30)
}

file_pairs = [
    ('Sujeto1_R_processed.fif', 'Sujeto1_MF_processed.fif'),
    ('Sujeto2_R_processed.fif', 'Sujeto2_MF_processed.fif'),
    ('Sujeto3_R_processed.fif', 'Sujeto3_MF_processed.fif'),
    ('Sujeto4_R_processed.fif', 'Sujeto4_MF_processed.fif'),
    ('Sujeto5_R_processed.fif', 'Sujeto5_MF_processed.fif'),
    ('Sujeto6_R_processed.fif', 'Sujeto6_MF_processed.fif'),
    ('Sujeto7_R_processed.fif', 'Sujeto7_MF_processed.fif'),
    ('Sujeto8_R_processed.fif', 'Sujeto8_MF_processed.fif'),
    ('Sujeto9_R_processed.fif', 'Sujeto9_MF_processed.fif'),
    ('Sujeto10_R_processed.fif', 'Sujeto10_MF_processed.fif'),
    ('Sujeto11_R_processed.fif', 'Sujeto11_MF_processed.fif'),
    #('Sujeto12_R_processed.fif', 'Sujeto12_MF_processed.fif'),  #always excluded
    ('Sujeto13_R_processed.fif', 'Sujeto13_MF_processed.fif'),
    ('Sujeto14_R_processed.fif', 'Sujeto14_MF_processed.fif'),
    ('Sujeto15_R_processed.fif', 'Sujeto15_MF_processed.fif'),
    ('Sujeto16_R_processed.fif', 'Sujeto16_MF_processed.fif'),
    ('Sujeto17_R_processed.fif', 'Sujeto17_MF_processed.fif'),
    ('Sujeto18_R_processed.fif', 'Sujeto18_MF_processed.fif'),
    ('Sujeto19_R_processed.fif', 'Sujeto19_MF_processed.fif'),
    ('Sujeto20_R_processed.fif', 'Sujeto20_MF_processed.fif'),
    ('Sujeto21_R_processed.fif', 'Sujeto21_MF_processed.fif'),
    ('Sujeto22_R_processed.fif', 'Sujeto22_MF_processed.fif'),
    ('Sujeto23_R_processed.fif', 'Sujeto23_MF_processed.fif'),
    ('Sujeto24_R_processed.fif', 'Sujeto24_MF_processed.fif'),
    ('Sujeto25_R_processed.fif', 'Sujeto25_MF_processed.fif'),
    ('Sujeto26_R_processed.fif', 'Sujeto26_MF_processed.fif'),
    ('Sujeto27_R_processed.fif', 'Sujeto27_MF_processed.fif'),
    ('Sujeto28_R_processed.fif', 'Sujeto28_MF_processed.fif'),
    ('Sujeto29_R_processed.fif', 'Sujeto29_MF_processed.fif'),
    ('Sujeto30_R_processed.fif', 'Sujeto30_MF_processed.fif'),
    ('Sujeto31_R_processed.fif', 'Sujeto31_MF_processed.fif'),
    ('Sujeto32_R_processed.fif', 'Sujeto32_MF_processed.fif'),
    # ('Sujeto33_R_processed.fif', 'Sujeto33_MF_processed.fif'),
    # ('Sujeto34_R_processed.fif', 'Sujeto34_MF_processed.fif'),
    #('Sujeto35_R_processed.fif', 'Sujeto35_MF_processed.fif'), #always excluded
    ('Sujeto36_R_processed.fif', 'Sujeto36_MF_processed.fif'),
    ('Sujeto37_R_processed.fif', 'Sujeto37_MF_processed.fif')
    ]

#responses = np.array([4, 4, 6, 7, 7, 1, 8, 7, 9, 2, 5, 7, 4, 6, 8, 6, 7, 7, 6, 8, 8, 4, 8, 9, 7, 8, 5, 7, 5, 4, 7, 4, 7, 6])  #AR2-MF
#responses = np.array([0, 3, 6, 1, 2, 4, 5, 6, 7, 1, 5, 8, 3, 1, 7, 4, 7, 8, 4, 4, 5, 1, 5, 2, 5, 6, 4, 1, 2, 1, 4, 0, 8, 1])  #AR3-MF
#responses = np.array([2, 5, 5, 5, 5, 3, 5, 6, 7, 1, 6, 4, 2, 7, 7, 3, 6, 4, 7, 5, 6, 3, 8, 9, 6, 5, 7, 5, 4, 3, 5, 1, 7, 6])  #AR4-MF
responses = np.array(responses_array.tolist())

# file_pairs=file_pairs[:5]
# responses=responses[:5]

# Call the function
topomap_correlation_states(file_pairs, responses, frequency_bands, title='Correlation with MWQT')


Initial counts - Sujetos: 36, Responses: 36

Eliminated due to NaN values:
31    SUJ_33
32    SUJ_34

Additional subjects being excluded: ['SUJ_12', 'SUJ_35']
These subjects will be removed:
33    SUJ_35

Final counts - Sujetos: 33, Responses: 33

Cleaned Sujetos:
0      SUJ_1
1      SUJ_2
2      SUJ_3
3      SUJ_4
4      SUJ_5
5      SUJ_6
6      SUJ_7
7      SUJ_8
8      SUJ_9
9     SUJ_10
10    SUJ_11
11    SUJ_13
12    SUJ_14
13    SUJ_15
14    SUJ_16
15    SUJ_17
16    SUJ_18
17    SUJ_19
18    SUJ_20
19    SUJ_21
20    SUJ_22
21    SUJ_23
22    SUJ_24
23    SUJ_25
24    SUJ_26
25    SUJ_27
26    SUJ_28
27    SUJ_29
28    SUJ_30
29    SUJ_31
30    SUJ_32
34    SUJ_36
35    SUJ_37

Cleaned Data (first 10 values):
0    17.0
1    18.0
2     8.0
3    10.0
4    22.0
5    24.0
6    13.0
7    14.0
8    12.0
9    20.0

NumPy array shape: (33,)
First 5 array values: [17. 18.  8. 10. 22.]
Reading c:\Users\afcad\Downloads\Sujeto1_R_processed.fif ...


C:\Users\afcad\AppData\Local\Temp\ipykernel_1044\2763569990.py:57: RuntimeWarning: This filename (Sujeto1_R_processed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  info = mne.read_epochs(file_pairs[0][0]).info


    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available
Not setting metadata
68 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\afcad\Downloads\Sujeto1_R_processed.fif ...


C:\Users\afcad\AppData\Local\Temp\ipykernel_1044\2763569990.py:69: RuntimeWarning: This filename (Sujeto1_R_processed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs_R = mne.read_epochs(file_R, preload=True)


    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available
Not setting metadata
68 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\afcad\Downloads\Sujeto1_MF_processed.fif ...


...

    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available
Not setting metadata
133 matching events found
No baseline correction applied
0 projection items activated
Effective window size : 0.250 (s)
Effective window size : 0.250 (s)
Reading c:\Users\afcad\Downloads\Sujeto37_R_processed.fif ...
    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available


C:\Users\afcad\AppData\Local\Temp\ipykernel_1044\2763569990.py:69: RuntimeWarning: This filename (Sujeto37_R_processed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs_R = mne.read_epochs(file_R, preload=True)


Not setting metadata
71 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\afcad\Downloads\Sujeto37_MF_processed.fif ...


C:\Users\afcad\AppData\Local\Temp\ipykernel_1044\2763569990.py:70: RuntimeWarning: This filename (Sujeto37_MF_processed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs_MF = mne.read_epochs(file_MF, preload=True)


    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available
Not setting metadata
141 matching events found
No baseline correction applied
0 projection items activated
Effective window size : 0.250 (s)
Effective window size : 0.250 (s)
Resting - Fast Beta (18–30 Hz): 0 significant channels (p < 0.05)
Mindfulness - Fast Beta (18–30 Hz): 0 significant channels (p < 0.05)
Difference (MF - R) - Fast Beta (18–30 Hz): 19 significant channels (p < 0.05)


In [20]:
import numpy as np
import mne
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import pandas as pd


# df = pd.read_excel('Analisis conductuales - Datos conductuales.xlsx')
df = pd.read_excel('EEG-Datos conductuales.xlsx')
#print(df.head())

sujeto_column = df['Sujeto']
data_column = df['MAIA'] #Auto-regulación Confianza Valencia positiva Valencia Negativa

print(f"Initial counts - Sujetos: {len(sujeto_column)}, Responses: {len(data_column)}")

# Create a mask where both columns are not NaN
mask = sujeto_column.notna() & data_column.notna()

eliminate_mask = sujeto_column.isna() | data_column.isna()
# Get the Sujeto values that will be removed
eliminated_sujetos = sujeto_column[eliminate_mask]

print("\nEliminated due to NaN values:")
print(eliminated_sujetos.to_string())  # to_string() shows all values

# Apply the initial mask to both columns
sujeto_clean = sujeto_column[mask]
data_clean = data_column[mask]

# Now exclude specific subjects
subjects_to_exclude = ['SUJ_12', 'SUJ_35']  # Add more as needed
additional_elim_mask = sujeto_clean.isin(subjects_to_exclude)

print(f"\nAdditional subjects being excluded: {subjects_to_exclude}")
print("These subjects will be removed:")
print(sujeto_clean[additional_elim_mask].to_string())

# Final clean mask
final_mask = ~additional_elim_mask
sujeto_final = sujeto_clean[final_mask]
data_final = data_clean[final_mask]

print(f"\nFinal counts - Sujetos: {len(sujeto_final)}, Responses: {len(data_final)}")
print("\nCleaned Sujetos:")
print(sujeto_final.to_string())
print("\nCleaned Data (first 10 values):")
print(data_final.head(10).to_string())

# Convert to numpy array
responses_array = data_final.to_numpy()
print(f"\nNumPy array shape: {responses_array.shape}")
print(f"First 5 array values: {responses_array[:5]}")

def topomap_correlation_states(file_pairs, responses, frequency_bands, title='Correlation with Questionnaire'):

    info = mne.read_epochs(file_pairs[0][0]).info
    n_bands = len(frequency_bands)
    n_channels = len(info['ch_names'])

    fig, axes = plt.subplots(3, n_bands, figsize=(3.5 * n_bands, 10))

    for col, (band_name, (fmin, fmax)) in enumerate(frequency_bands.items()):
        psd_R_all = []
        psd_MF_all = []

        # --- Extract and average PSDs ---
        for file_R, file_MF in file_pairs:
            epochs_R = mne.read_epochs(file_R, preload=True)
            epochs_MF = mne.read_epochs(file_MF, preload=True)

            psd_R, freqs = epochs_R.compute_psd(method='welch', fmin=fmin, fmax=fmax).get_data(return_freqs=True)
            psd_MF, _ = epochs_MF.compute_psd(method='welch', fmin=fmin, fmax=fmax).get_data(return_freqs=True)

            avg_psd_R = np.mean(psd_R, axis=(0, 2))  # shape: (n_channels,)
            avg_psd_MF = np.mean(psd_MF, axis=(0, 2))

            psd_R_all.append(avg_psd_R)
            psd_MF_all.append(avg_psd_MF)

        psd_R_all = np.array(psd_R_all)      # shape: (n_subjects, n_channels)
        psd_MF_all = np.array(psd_MF_all)
        psd_diff = psd_MF_all - psd_R_all

        # --- Correlation for each condition ---
        for row, (state_name, data) in enumerate([
            ("Resting", psd_R_all),
            ("Mindfulness", psd_MF_all),
            ("Difference (MF - R)", psd_diff)
        ]):
            r_vals = np.zeros(n_channels)
            p_vals = np.zeros(n_channels)

            for ch in range(n_channels):
                r_vals[ch], p_vals[ch] = pearsonr(data[:, ch], responses)

            sig_mask = p_vals < 0.05  # Threshold for significance

            print(f'{state_name} - {band_name}: {np.sum(sig_mask)} significant channels (p < 0.05)')

            im, cn = mne.viz.plot_topomap(
                r_vals, info, axes=axes[row, col], cmap='jet', show=False,
                names=info['ch_names'],
                vlim=(-1, 1),
                mask=sig_mask,
                mask_params=dict(marker='*', markersize=25, markerfacecolor='black')
            )
            axes[row, col].set_title(f'{band_name}\n{state_name}', fontsize=10)
            plt.colorbar(im, ax=axes[row, col], orientation='horizontal', shrink=0.6)

    # --- Row Labels ---
    for ax, label in zip(axes[:, 0], ['Resting', 'Mindfulness', 'Difference']):
        ax.annotate(label, xy=(-0.2, 0.5), xycoords='axes fraction',
                    va='center', ha='right', rotation=90, fontsize=12)

    plt.suptitle(title, fontsize=16)
    plt.tight_layout()
    plt.show()


# Your frequency bands
frequency_bands = {
    'Delta (0.5–4 Hz)': (0.5, 4),
    'Theta (4–8 Hz)': (4, 8),
    'Alpha (8–12 Hz)': (8, 12),
    'Beta (12–18 Hz)': (12, 18),
    'Fast Beta (18–30 Hz)': (18, 30)
}

file_pairs = [
    ('Sujeto1_R_processed.fif', 'Sujeto1_MF_processed.fif'),
    ('Sujeto2_R_processed.fif', 'Sujeto2_MF_processed.fif'),
    ('Sujeto3_R_processed.fif', 'Sujeto3_MF_processed.fif'),
    ('Sujeto4_R_processed.fif', 'Sujeto4_MF_processed.fif'),
    ('Sujeto5_R_processed.fif', 'Sujeto5_MF_processed.fif'),
    ('Sujeto6_R_processed.fif', 'Sujeto6_MF_processed.fif'),
    ('Sujeto7_R_processed.fif', 'Sujeto7_MF_processed.fif'),
    ('Sujeto8_R_processed.fif', 'Sujeto8_MF_processed.fif'),
    ('Sujeto9_R_processed.fif', 'Sujeto9_MF_processed.fif'),
    ('Sujeto10_R_processed.fif', 'Sujeto10_MF_processed.fif'),
    ('Sujeto11_R_processed.fif', 'Sujeto11_MF_processed.fif'),
    #('Sujeto12_R_processed.fif', 'Sujeto12_MF_processed.fif'),  #always excluded
    ('Sujeto13_R_processed.fif', 'Sujeto13_MF_processed.fif'),
    ('Sujeto14_R_processed.fif', 'Sujeto14_MF_processed.fif'),
    ('Sujeto15_R_processed.fif', 'Sujeto15_MF_processed.fif'),
    ('Sujeto16_R_processed.fif', 'Sujeto16_MF_processed.fif'),
    ('Sujeto17_R_processed.fif', 'Sujeto17_MF_processed.fif'),
    ('Sujeto18_R_processed.fif', 'Sujeto18_MF_processed.fif'),
    ('Sujeto19_R_processed.fif', 'Sujeto19_MF_processed.fif'),
    ('Sujeto20_R_processed.fif', 'Sujeto20_MF_processed.fif'),
    ('Sujeto21_R_processed.fif', 'Sujeto21_MF_processed.fif'),
    ('Sujeto22_R_processed.fif', 'Sujeto22_MF_processed.fif'),
    ('Sujeto23_R_processed.fif', 'Sujeto23_MF_processed.fif'),
    ('Sujeto24_R_processed.fif', 'Sujeto24_MF_processed.fif'),
    ('Sujeto25_R_processed.fif', 'Sujeto25_MF_processed.fif'),
    ('Sujeto26_R_processed.fif', 'Sujeto26_MF_processed.fif'),
    ('Sujeto27_R_processed.fif', 'Sujeto27_MF_processed.fif'),
    ('Sujeto28_R_processed.fif', 'Sujeto28_MF_processed.fif'),
    ('Sujeto29_R_processed.fif', 'Sujeto29_MF_processed.fif'),
    ('Sujeto30_R_processed.fif', 'Sujeto30_MF_processed.fif'),
    ('Sujeto31_R_processed.fif', 'Sujeto31_MF_processed.fif'),
    ('Sujeto32_R_processed.fif', 'Sujeto32_MF_processed.fif'),
    # ('Sujeto33_R_processed.fif', 'Sujeto33_MF_processed.fif'),
    # ('Sujeto34_R_processed.fif', 'Sujeto34_MF_processed.fif'),
    #('Sujeto35_R_processed.fif', 'Sujeto35_MF_processed.fif'), #always excluded
    ('Sujeto36_R_processed.fif', 'Sujeto36_MF_processed.fif'),
    ('Sujeto37_R_processed.fif', 'Sujeto37_MF_processed.fif')
    ]

#responses = np.array([4, 4, 6, 7, 7, 1, 8, 7, 9, 2, 5, 7, 4, 6, 8, 6, 7, 7, 6, 8, 8, 4, 8, 9, 7, 8, 5, 7, 5, 4, 7, 4, 7, 6])  #AR2-MF
#responses = np.array([0, 3, 6, 1, 2, 4, 5, 6, 7, 1, 5, 8, 3, 1, 7, 4, 7, 8, 4, 4, 5, 1, 5, 2, 5, 6, 4, 1, 2, 1, 4, 0, 8, 1])  #AR3-MF
#responses = np.array([2, 5, 5, 5, 5, 3, 5, 6, 7, 1, 6, 4, 2, 7, 7, 3, 6, 4, 7, 5, 6, 3, 8, 9, 6, 5, 7, 5, 4, 3, 5, 1, 7, 6])  #AR4-MF
responses = np.array(responses_array.tolist())

# file_pairs=file_pairs[:5]
# responses=responses[:5]

# Call the function
topomap_correlation_states(file_pairs, responses, frequency_bands, title='Correlation with MWQT')


Initial counts - Sujetos: 36, Responses: 36

Eliminated due to NaN values:
31    SUJ_33
32    SUJ_34

Additional subjects being excluded: ['SUJ_12', 'SUJ_35']
These subjects will be removed:
33    SUJ_35

Final counts - Sujetos: 33, Responses: 33

Cleaned Sujetos:
0      SUJ_1
1      SUJ_2
2      SUJ_3
3      SUJ_4
4      SUJ_5
5      SUJ_6
6      SUJ_7
7      SUJ_8
8      SUJ_9
9     SUJ_10
10    SUJ_11
11    SUJ_13
12    SUJ_14
13    SUJ_15
14    SUJ_16
15    SUJ_17
16    SUJ_18
17    SUJ_19
18    SUJ_20
19    SUJ_21
20    SUJ_22
21    SUJ_23
22    SUJ_24
23    SUJ_25
24    SUJ_26
25    SUJ_27
26    SUJ_28
27    SUJ_29
28    SUJ_30
29    SUJ_31
30    SUJ_32
34    SUJ_36
35    SUJ_37

Cleaned Data (first 10 values):
0    18.073810
1    22.057143
2    21.876190
3     9.350000
4    17.845238
5    13.371429
6    28.316667
7    27.407143
8    29.033333
9    23.504762

NumPy array shape: (33,)
First 5 array values: [18.07380952 22.05714286 21.87619048  9.35       17.8452381 ]
Reading c:\Us

C:\Users\afcad\AppData\Local\Temp\ipykernel_1044\2538836546.py:57: RuntimeWarning: This filename (Sujeto1_R_processed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  info = mne.read_epochs(file_pairs[0][0]).info


        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available
Not setting metadata
68 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\afcad\Downloads\Sujeto1_R_processed.fif ...
    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available


C:\Users\afcad\AppData\Local\Temp\ipykernel_1044\2538836546.py:69: RuntimeWarning: This filename (Sujeto1_R_processed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs_R = mne.read_epochs(file_R, preload=True)


Not setting metadata
68 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\afcad\Downloads\Sujeto1_MF_processed.fif ...


...

    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available
Not setting metadata
133 matching events found
No baseline correction applied
0 projection items activated
Effective window size : 0.250 (s)
Effective window size : 0.250 (s)
Reading c:\Users\afcad\Downloads\Sujeto37_R_processed.fif ...
    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available


C:\Users\afcad\AppData\Local\Temp\ipykernel_1044\2538836546.py:69: RuntimeWarning: This filename (Sujeto37_R_processed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs_R = mne.read_epochs(file_R, preload=True)


Not setting metadata
71 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\afcad\Downloads\Sujeto37_MF_processed.fif ...


C:\Users\afcad\AppData\Local\Temp\ipykernel_1044\2538836546.py:70: RuntimeWarning: This filename (Sujeto37_MF_processed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs_MF = mne.read_epochs(file_MF, preload=True)


    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available
Not setting metadata
141 matching events found
No baseline correction applied
0 projection items activated
Effective window size : 0.250 (s)
Effective window size : 0.250 (s)
Resting - Fast Beta (18–30 Hz): 0 significant channels (p < 0.05)
Mindfulness - Fast Beta (18–30 Hz): 0 significant channels (p < 0.05)
Difference (MF - R) - Fast Beta (18–30 Hz): 6 significant channels (p < 0.05)
